# Сопроводительный ноутбук к заданию №2

**Оглавление** <a id='toc'></a>

1. [Рекуррентные нейронные сети](#section_rnn)
    1. [Векторные представления слов](#ssection_embeddings)
    2. [Функционирование RNN](#ssection_rnn_functioning)
    3. [Внутреннее устройство RNN ячеек](#ssection_rnn_cells)
        1. [Ячейка "RNN" (`torch.nn.RNNCell`)](#sssection_rnn_cell)
        2. [Ячейка "Управляемый рекуррентный блок" (GRU, `torch.nn.GRUCell`)](#sssection_gru_cell)
        3. [Ячейка "Долгая краткосрочная память" (LSTM, `torch.nn.LSTMCell`)](#sssection_lstm_cell)
    4. [RNN-ячейки в `torch`](#ssection_torch_rnn_cells)
    5. [RNN слои в `torch`](#ssection_torch_rnn)
    6. [Пример использования RNN слоев в `torch`](#ssection_torch_rnn_example)
    7. [Упаковка последовательностей разной длины](#ssection_sequences_packing)
        1. [Упаковка последовательности](#sssection_sequences_packing_filtration)
            1. [Несколько трюков](#ssssection_sequences_packing_filtration_tricks)
        2. [Уравнивание длин последовательностей (метод набивки)](#sssection_sequences_packing_padding)
        3. [Реализация подходов упаковки и набивки в `torch`](#sssection_sequences_packing_torch)
            1. ["Чистый" метод упаковки](#ssssection_sequences_packing_torch_packing)
            2. ["Чистый" метод набивки](#ssssection_sequences_packing_torch_padding)
            3. [Совмещение обоих подходов](#ssssection_sequences_packing_torch_mixed)
2. [Трансформеры](#section_transformers)
    1. [Self Attention (SA)](#ssection_transformers_SA)
    2. [Multi-head Self-Attention (MHSA)](#ssection_transformers_MHSA)
3. [Специальные токены](#section_special_tokens)

# Рекуррентные нейронные сети (RNN) <a id='section_rnn'></a>
**Рекуррентные нейронные сети** (**RNN**) — вид нейронных сетей, предназначенных для обработки последовательностей произвольной длины, поэтому с помощью них можно решать задачи, связанные с текстами и временными рядами. В рамках домашнего задания вам будет предложено поработать с текстами.

[К оглавлению](#toc)

## Векторные представления слов <a id='ssection_embeddings'></a>
Для того, чтобы подать последовательность слов $t_1, t_2, \dots, t_T$ на вход RNN, нужно сначала каждое слово перевести в его **эмбеддинг** или, по-русски, в **векторное представление**.
Для этого сначала составляют множество слов (**словарь**) $W$, для которых будут обучаться эмбеддинги. Все слова во входной последовательности, не попавшие в словарь, будут обработаны особым способом. Далее каждому слову $w \in W$ сопоставляется уникальный индекс от $0$ до $|W| - 1$ включительно. После этого используют дополнительный слой `torch.nn.Embedding`, который как раз и делает основную работу — переводит индекс слова из словаря в соответствующий ему обучаемый эмбеддинг. Размерность эмбеддингов будем обозначать за $d_{in}$. Обычное значения для $d_{in}$ — порядка сотен: 50, 100, 200, 300.

Рассмотрим пример, в котором есть два входных текста:

In [1]:
texts = [
    ["you", "are", "my", "friend"],
    ["dear", "friend", "where", "are", "you"]
]

В обработке естественного языка (Natural Language Processing, NLP) множество всех или части интересующих нас слов, используемых в наборе данных, принято называть словарем (vocabulary).
Построим словарь из всех слов из примера:

In [2]:
vocabulary = {
    word
    for text in texts
    for word in text
}

vocabulary

{'are', 'dear', 'friend', 'my', 'where', 'you'}

Построим отображение слов из словаря в индексы:

In [3]:
word2ind = {
    word: index
    for index, word
    in enumerate(vocabulary)
}

word2ind

{'are': 0, 'you': 1, 'my': 2, 'where': 3, 'friend': 4, 'dear': 5}

Отобразим слова в текстах в соответствующие им индексы:

In [4]:
idxs = [
    [
        word2ind[word]
        for word in text
    ]
    for text in texts
]

idxs

[[1, 0, 2, 4], [5, 4, 3, 0, 1]]

Создадим слой `torch.nn.Embedding` для перевода индексов в векторные представления (для примера положим $d_{in}$ равным 2):

In [5]:
import torch          # Импортируем библиотеку torch,
from torch import nn  #  а также модуль nn для работы с нейронными сетями

d_in = 2

torch.manual_seed(42)  # Для воспроизводимости инициализации эмбеддингов
embeddings = nn.Embedding(num_embeddings=len(vocabulary), embedding_dim=d_in)

Посмотрим на то, как теперь выглядит каждый из текстов:

In [6]:
header = f"{'Слово':7} -> {'Индекс':6} -> {'Эмбеддинг':26}"
print(header)
print('-' * len(header))

for text, text_idxs in zip(texts, idxs):
    # Получаем эмбеддинги по индексам слов
    embeds = embeddings(torch.tensor(text_idxs))

    # Печатаем слова, индексы и эмбеддинги
    for word, idx, embedding in zip(text, text_idxs, embeds):
        print(f"{word:7} -> {idx:6} -> {embedding.data}")
    print('-' * len(header))

Слово   -> Индекс -> Эмбеддинг                 
-----------------------------------------------
you     ->      1 -> tensor([0.2345, 0.2303])
are     ->      0 -> tensor([0.3367, 0.1288])
my      ->      2 -> tensor([-1.1229, -0.1863])
friend  ->      4 -> tensor([0.4617, 0.2674])
-----------------------------------------------
dear    ->      5 -> tensor([0.5349, 0.8094])
friend  ->      4 -> tensor([0.4617, 0.2674])
where   ->      3 -> tensor([ 2.2082, -0.6380])
are     ->      0 -> tensor([0.3367, 0.1288])
you     ->      1 -> tensor([0.2345, 0.2303])
-----------------------------------------------


Как можно заметить, одному и тому же слову в разных предложениях соответствует один и тот же эмбеддинг.

На практике обычно не обучают векторные представления сразу для всех слов из датасета, потому что уникальных слов может быть очень много (сотни тысяч), из-за чего словарь будет большим и, как следствие, слой `torch.nn.Embedding` будет занимать много памяти.
Например, для словаря из $500 000$ слов и размерности векторных представлений $300$ понадобится порядка $(500000 \cdot 300 \cdot 8)/2^{30} \approx 1.12$ Гб памяти. На современных пользовательских GPU обычно имеется порядка 2-11 Гб видео памяти, поэтому каждый Гб, занимаемый вашим приложением, будет на счету.

Также обучать эмбеддинги для редких слов, которых обычно много, практически бессмысленно, потому что из-за нехватки обучающих данных векторные представления для них будут малоинформативными.

К тому же во время инференса модели (то есть использования модели уже после обучения для получения предсказаний) все равно могут попасться на вход какие-нибудь неизвестные ей слова. Тут уже не получится справиться с этой проблемой расширением словаря, потому что у нас нет обучающих данных с этими неизвестными словами.

Также можно использовать в качестве эмбеддингов предобученные представления слов, например, [word2vec](https://huggingface.co/fse/word2vec-google-news-300), [Glove](https://nlp.stanford.edu/projects/glove/), [FastText](https://fasttext.cc/docs/en/crawl-vectors.html).

Со словами, не попавшими в словарь, можно поступить несколькими способами:
1. Удалить все неизвестные слова;
2. Заменить каждое такое слово на специальное слово "`<UNK>`";
3. Составлять словарь не из слов, а из небольших частей слов.

Если с первым пунктом все понятно, то оставшиеся требуют пояснений.  
Во втором способе предлагается в словарь добавить специальное слово "`<UNK>`" и для него обучать векторное представление. Во входной последовательности $t_1, \dots, t_T$ до перевода ее в последовательность эмбеддингов все неизвестные слова заменяются на слово "`<UNK>`", после чего перевод слов в эмбеддинги продолжается обычным способом.
В специальном слове "`<UNK>`" используются треугольные скобки для того, чтобы не путать это слово с неспециальными словами.  
Про последний способ подробно говорить в этом материале не будем.

Далее будем считать, что входную последовательность слов $t_1, t_2, \dots, t_T$ мы уже перевели в последовательность соответствующих им эмбеддингов $x_1, x_2, \dots, x_T$.

**Резюме главы**
1. RNN позволяют решать задачи, в которых длина входа произвольна.
2. Входные слова перед подачей в RNN надо превратить в эмбеддинги. Это можно сделать с помощью `torch.nn.Embedding` слоя.
3. Нужно как-то бороться с проблемой неизвестных слов.

[К оглавлению](#toc)

## Функционирование RNN <a id='ssection_rnn_functioning'></a>
Высокоуровнево функционирование одного слоя RNN выглядит следующим образом:

![Unrolled RNN](images/rnn-unrolled.png "Unrolled RNN")

RNN слой на рисунке обозначен буквой А. В этом слое есть какие-то обучающиеся параметры, благодаря которым RNN как раз и обучается. В нашем случае RNN слой принимает на вход последовательность $x_1, x_2, \dots, x_T \in \mathbb{R}^{d_{in}}$ (на рисунке нумерация индексов начинается с 0, но мы в тексте будем придерживаться нумерации с 1). Справа на рисунке изображен один и тот же слой А для того, чтобы продемонстрировать, как он функционирует.

Для обработки каждой отдельной входной последовательности $x_1, x_2, \dots, x_T \in \mathbb{R}^{d_{in}}$ в RNN заводится внутреннее состояние $h \in \mathbb{R}^{d_{hidden}}$, которое обычно инициализируется случайно или нулями.

Далее через RNN начинают пропускать входные эмбеддинги по одному. На шаге $t = \overline{1, T}$ эмбеддинг $x_t$ подается в RNN. RNN внутри себя пережевывает $x_t$ и $h$, после чего обновляет внутреннее состояние $h$ и выдает выходной эмбеддинг $y_t \in \mathbb{R}^{d_{hidden}}$. Состояние сети после шага $t$ будем обозначать за $h_t$.

На рисунке изображено, что сеть в качестве выхода выдает внутреннее состояния $h_t$ без изменения, то есть в этом случае полагается $y_t = h_t$. 
Обычно так и делают, хотя ничего не мешает как-нибудь преобразовать $h_t$ перед выдачей из сети.

После того, как вся последовательность $x_1, \dots, x_T$ была пропущена через RNN, имеем на выходе из сети последовательность $y_1, \dots, y_T \in \mathbb{R}^{d_{hidden}}$.

Уже такая модель (пока что мы не специфицировали, что модель делает конкретно внутри себя, но это мы опишем чуть позже) может решать задачи машинного обучения. Как именно — зависит от самой задачи. Например, для классификации входного предложения $t_1, \dots, t_T$ целиком как единой сущности можно обучать MLP-классификатор (MLP — Multilayer Perceptron, Многослойный Перцептрон — полносвязная нейронная сеть) поверх последнего выходного вектора $y_T$, что логично, так как в $y_T$ находится информация о всем входном тексте. Для разметки каждого слова последовательности, например, в задачах определения частей речи (PoS) или выделения именованных сущностей (NER), можно обучить MLP-классификатор на каждом выходном векторе $y_t$.

Слой RNN, который мы описали выше, обычно называют не слоем, а **ячейкой** (**cell**, **RNN cell**). Будем и дальше придерживаться такой терминологии, а под слоем будет понимать более сложную вещь, речь о которой пойдет дальше.

Для некоторых задач такая простая модель может показывать не очень высокое качество, поэтому есть несколько способов ее усложнить.

Как можно понять, выходной вектор $y_t$ зависит только от первых $t$ слов предложения: от $t_1, t_2, \dots, t_t$. В случае решения задачи NER такого **левостороннего контекста** может оказаться недостаточно для хорошего качества модели. Для того, чтобы в $y_t$ учесть еще и **правосторонний контекст**, обычно добавляют слой RNN, описанный выше, только в него вход $x_1, \dots, x_T$ подается задом наперед: сначала подается $x_T$, затем $x_{T-1}, x_{T-2}, \dots, x_2$ и только в самом конце $x_1$. Обозначим выход изначальной RNN, в которую вход подавался слева направо, за $\overrightarrow{y_1}, \overrightarrow{y_2}, \dots, \overrightarrow{y_T}$, а выход второй RNN за $\overleftarrow{y_1}, \overleftarrow{y_2}, \dots, \overleftarrow{y_T}$. Тогда выход $y_t \in \mathbb{R}^{2 \cdot d_{hidden}}$ этих двух слоев получим с помощью конкатенации векторов $\overrightarrow{y_t}$ и $\overleftarrow{y_t}$.
Теперь в $y_t$ учитывается двусторонний контекст, а такая модель должна уже лучше справится с задачей NER. Такую RNN называют **двунаправленной** (**bidirectional**).

![BiRNN layer](images/birnn-layer.png "BiRNN layer")
На рисунке в качестве А обозначена RNN, читающая вход слева-направо, а за А' — RNN, читающая вход справа-налево. Скрытые состояния обозначены буквами $s$ и $s'$.

Можно еще пуще усложнить полученную модель. Как видим, RNN на вход приняла последовательность векторов ($x_1, \dots, x_T$) и выдала на выход последовательность векторов ($y_1, \dots, y_T$). Тогда можно создать еще один такой же слой RNN и пропустить через него $y_1, \dots, y_T$. Новый слой RNN будет отличаться единственным изменением размерности входных векторов — первый слой RNN принимал на вход вектора размерности $d_{in}$, а сейчас мы хотим подавать на вход вектора размерности $d_{hidden}$ или $2 \cdot d_{hidden}$ в зависимости от направленности сети. Пусть выходом нового слоя станет последовательность $z_1, \dots, z_T \in \mathbb{R}^{d_{hidden}}$ (или $z_1, \dots, z_T \in \mathbb{R}^{2 \cdot d_{hidden}}$ в случае двунаправленного слоя). Мы с ней дальше можем делать все то же самое, что могли делать с последовательностью $y_1, \dots, y_T$, в том числе и решать вышеописанные задачи.

Таким образом можно составить RNN из произвольно большого числа слоев. Однако, на практике редко используют больше 2-3 слоев из-за проблем обучения таких глубоких сетей:
- их тяжело обучать из-за нестабильности градиентов — они могут либо взрываться, либо обращаться в ноль,
- большое число параметров в модели делает глубокие сети очень ресурсоемкими.

![Multi-layer RNN](images/multilayer-rnn.svg "Multi-layer RNN")
Рисунок RNN с несколькими слоями. Тут за $H_t^{(l)}$ обозначен выход (и внутреннее состояние) RNN слоя с номером $l$ после обработки $t$-го элемента входной последовательности.

Стоит добавить, что в каждом отдельном слое RNN поддерживается свое независимое внутреннее состояние $h$. Отдельными слоями RNN являются слои разных направлений (слева-направо и справа-налево), а также слои, находящиеся на разных уровнях (например, слой, превращающий $x$ в $y$ и слой, превращающий $y$ в $z$, находятся на разных уровнях).

Кстати, двунаправленные RNN иногда нет возможности использовать. Например, в задаче генерации текста после вновь сгенерированного слова пришлось бы перезапускать RNN-ячейку-справа-налево заново из-за необходимости учета нового слова. К тому же особого смысла эмбеддинги, получаемые этой ячейкой, не имели бы, постольку поскольку эмбеддинг $\overrightarrow{y_T}$ будет иметь в себе информацию о всей последовательности.

**Резюме главы**
1. В RNN есть внутренние состояния, а сама сеть пропускает через себя входные эмбеддинги по одному.
2. Выходом RNN является последовательность такой же длины, что и входная.
3. RNN можно усложнить, сделав ее двунаправленной, но это не всегда возможно.
4. RNN можно усложнить, составив в ней несколько слоев (на практике от 1 до 3 слоев).
5. RNN слой состоит из RNN-ячеек.

[К оглавлению](#toc)

## Внутреннее устройство RNN ячеек <a id='ssection_rnn_cells'></a>
Как мы поняли, RNN слой состоит из нескольких (может быть и одной) RNN-ячеек. Теперь для полноты картины осталось понять, что происходит внутри ячейки RNN. Понятно, что внутри ячейки могут производится произвольные вычисления. На практике обычно используют 3 варианта реализации рекуррентной ячейки. Все эти реализации есть и в модуле `torch.nn`.

Посмотрим, что происходит внутри ячейки с внутренним состоянием $h$ при обработке входного эмбеддинга $x$.

На уже знакомом нам рисунке RNN-ячейка обозначена за $A$.
![Unrolled RNN](images/rnn-unrolled.png "Unrolled RNN")

### Ячейка "RNN" (`torch.nn.RNNCell`) <a id='sssection_rnn_cell'></a>
Данная ячейка реализована в виде класса `torch.nn.RNNCell`.

Имеет 4 обучаемых параметра: 2 матрицы $W_{ih} \in \mathbb{R}^{d_{hidden} \times d_{in}}$, $W_{hh} \in \mathbb{R}^{d_{hidden} \times d_{hidden}}$ и 2 вектора сдвигов (bias) $b_{ih} \in \mathbb{R}^{d_{hidden}}$, $b_{hh} \in \mathbb{R}^{d_{hidden}}$.  
Матрица $W_{ih}$ и сдвиг $b_{ih}$ используются для трансформации входного вектора $x$, а матрица $W_{hh}$ и сдвиг $b_{hh}$ используются для трансформации внутреннего состояния ячейки $h$.  
На выходе из ячейки имеем вектор $h^{\prime}$ — новое внутренне состояние ячейки.

Преобразование в ячейке описывается следующим выражением:

$$
h^{\prime}=\tanh \left(W_{i h} x+b_{i h}+W_{h h} h+b_{h h}\right).
$$

По сути к входному вектору и вектору внутреннего состояния ячейки применяются афинные преобразования, результаты преобразований складываются, после чего к итоговому вектору применяется нелинейность $\tanh(x) = \frac{e^{x} - e^{-x}}{e^{x} + e^{-x}}$.

В качестве функции активации используется $\tanh$ для того, чтобы добавить нелинейность в преобразование, а также чтобы значения выхода ячейки оказывались по модулю не больше 1 для стабильности градиентов и обучения.

### Ячейка "Управляемый рекуррентный блок" (GRU, `torch.nn.GRUCell`) <a id='sssection_gru_cell'></a>
Данная ячейка реализована в виде класса `torch.nn.GRUCell`.

Имеет 12 обучаемых параметров:
- 3 матрицы для преобразования входного вектора $x$: $W_{ir}, W_{iz}, W_{in} \in \mathbb{R}^{d_{hidden} \times d_{in}}$;
- 3 матрицы для преобразования вектора внутреннего состояния ячейки $h$: $W_{hr}, W_{hz}, W_{hn} \in \mathbb{R}^{d_{hidden} \times d_{hidden}}$;
- 3 вектора сдвигов (bias) для преобразования входного вектора $x$: $b_{ir}, b_{iz}, b_{in} \in \mathbb{R}^{d_{in}}$;
- 3 вектора сдвигов (bias) для преобразования вектора внутреннего состояния ячейки $h$: $b_{hr}, b_{hz}, b_{hn} \in \mathbb{R}^{d_{hidden}}$.

Матрицы $W_{i ?}$ и сдвиги $b_{i ?}$ используются для трансформации входного вектора $x$, а матрицы $W_{h ?}$ и сдвиги $b_{h ?}$ используются для трансформации внутреннего состояния ячейки $h$. Здесь под знаком вопроса обозначена любая из букв $r, z$ и $n$.  
На выходе из ячейки имеем вектор $h^{\prime}$.

Преобразования в ячейке описываются следующими выражениями:

$$
\begin{aligned}
&r=\sigma\left(W_{i r} x+b_{i r}+W_{h r} h+b_{h r}\right) \\
&z=\sigma\left(W_{i z} x+b_{i z}+W_{h z} h+b_{h z}\right) \\
&n=\tanh \left(W_{i n} x+b_{i n}+r *\left(W_{h n} h+b_{h n}\right)\right) \\
&h^{\prime}=(1-z) * n+z * h
\end{aligned}
$$

![GRU cell](images/gru-cell.png "GRU cell")

На схеме вектор $n$ обозначен за $\tilde{h_t}$.

Сначала в ячейке вычисляются два вектора $r$ и $z$. Они получаются аналогично тому, как в `RNNCell` вычисляется выход $h^{\prime}$, только в конце применяется не $\tanh$, а сигмоида $\sigma(x) = \frac{1}{1 + \exp(-x)}$. Смысл использования сигмоиды тут для получения значений в отрезке от 0 до 1 для использования в качестве весов для последующих взвешенных сумм.

Далее $z$ используется при расчете вектора $n$ — он вычисляется аналогично векторам $r$ и $z$ за исключением того, что преобразованное скрытое состояние домножается поточечно на вектор $r$ (а в конце применяется $\tanh$, а не сигмоида). Таким образом, если все компоненты вектора $r$ будут близки к 0, то в векторе $n$ почти не будет никакой информации о прошлом. Если же компоненты вектора $r$ будут близки к 1, то в векторе $n$ информация о прошлом сохранится. По сути $z$ помогает модели определить, сколько уже выученной ячейкой информации нужно сохранить в векторе $n$.

Далее выходом модели считается взвешенная сумма векторов $n$ и $h$ с коэффициентами $z$. Чем ближе компоненты вектора $z$ к 1, тем больше модель сохраняет свое старое состояние. Чем ближе компоненты вектора $z$ к 0, тем больше модель склонна к тотальному обновлению своего внутреннего состояния с $h$ на $n$.

Во время работы ячейки 6 афинных преобразований (3 для $x$ и 3 для $h$) применяются к одним и тем же векторам. Для эффективности вычислений можно 3 афинных преобразования над вектором $x$ (аналогично и для вектора $h$) произвести одновременно с помощью одного матричного произведения вместо трех (подумайте, почему так будет быстрее!). Для этого:
- Матрицы $W_{i?}$ объединяют в одну $W_{ih} \in \mathbb{R}^{3d_{hidden} \times d_{in}}$;
- Матрицы $W_{h?}$ объединяют в одну $W_{hh} \in \mathbb{R}^{3d_{hidden} \times d_{hidden}}$;
- Векторы $b_{i?}$ объединяют в один $b_{ih} \in \mathbb{R}^{3d_{hidden}}$;
- Векторы $b_{h?}$ объединяют в один $b_{hh} \in \mathbb{R}^{3d_{hidden}}$;
- Во время работы ячейки к входу $x$ и внутреннему состоянию $h$ применяются сразу все афинные преобразования с помощью матричного произведения $q$ = $W_{ih} x + b_{ih} + W_{hh} h + b_{hh}$.
- После этого вектор $q \in \mathbb{R}^{3 d_{hidden}}$ разрезается на три части $r$, $z$, $n$. Все оставшиеся вычисления выполняются по формулам.

В реализации `torch.nn.GRUCell` для эффективности вычислений применяется именно этот трюк.

Подробнее про GRU сети можно [почитать тут](https://towardsdatascience.com/understanding-gru-networks-2ef37df6c9be).

### Ячейка "Долгая краткосрочная память" (LSTM, `torch.nn.LSTMCell`) <a id='sssection_lstm_cell'></a>
Данная ячейка реализована в виде класса `torch.nn.LSTMCell`.

Имеет 16 обучаемых параметров:
- 4 матрицы для преобразования входного вектора $x$: $W_{ii}, W_{if}, W_{ig}, W_{io} \in \mathbb{R}^{d_{hidden} \times d_{in}}$;
- 4 матрицы для преобразования вектора внутреннего состояния ячейки $h$: $W_{hi}, W_{hf}, W_{hg}, W_{ho} \in \mathbb{R}^{d_{hidden} \times d_{hidden}}$;
- 4 вектора сдвигов (bias) для преобразования входного вектора $x$: $b_{ii}, b_{if}, b_{ig}, b_{io} \in \mathbb{R}^{d_{in}}$;
- 4 вектора сдвигов (bias) для преобразования вектора внутреннего состояния ячейки $h$: $b_{hi}, b_{hf}, b_{hg}, b_{ho} \in \mathbb{R}^{d_{hidden}}$.

Матрицы $W_{i ?}$ и сдвиги $b_{i ?}$ используются для трансформации входного вектора $x$, а матрицы $W_{h ?}$ и сдвиги $b_{h ?}$ используются для трансформации внутреннего состояния ячейки $h$. Здесь под знаком вопроса обозначена любая из букв $i, f, g$ и $o$.  
На выходе из ячейки имеем вектор $h^{\prime}$.

Преобразования в ячейке описываются следующими выражениями:

$$
\begin{aligned}
&i=\sigma\left(W_{i i} x+b_{i i}+W_{h i} h+b_{h i}\right) \\
&f=\sigma\left(W_{i f} x+b_{i f}+W_{h f} h+b_{h f}\right) \\
&g=\tanh \left(W_{i g} x+b_{i g}+W_{h g} h+b_{h g}\right) \\
&o=\sigma\left(W_{i o} x+b_{i o}+W_{h o} h+b_{h o}\right) \\
&c^{\prime}=f * c+i * g \\
&h^{\prime}=o * \tanh \left(c^{\prime}\right)
\end{aligned}
$$

![LSTM cell](images/lstm-cell.png "LSTM cell")

В `LSTMCell` в отличие от `RNNCell` и `GRUCell`, используется дополнительный вектор внутреннего состояния $c$, отвечающий за долговременную память ячейки. Вектор $h$ называют краткосрочной памятью ячейки LSTM. По сути в ячейке LSTM за внутреннее состояние теперь отвечают два вектора — $h$ и $c$.

Сначала в ячейке вычисляются четыре вектора $i, f, g$ и $o$. Они получаются аналогично тому, как в `RNNCell` вычисляется выход $h^{\prime}$, только в конце применяется не $\tanh$, а сигмоида для вычисления $i, f$ и $o$. Вектор $g$ считается кандидатом на новую долговременную память ячейки.

Далее новая долговременная память $c^{\prime}$ получается из прошлой долговременной памяти $c$ и кандидата $g$ с помощью взвешенной суммы с коэффициентами $f$ при $c$ и $i$ при $g$.  
$f$ называют "forget gate" или вентилем забывания (на мой взгляд было бы логичнее называть его вентилем запоминания). Он отвечает за то, что перекочует из старой долгосрочной памяти в новую.  
$i$ называют "input gate" или вентилем входа. Он отвечает за то, что добавится в новую долгосрочную память из кандидата $g$.

Новое значение внутреннего состояния $h$ ячейки образуется поточечным домножением на $o$ вектора $h^{\prime}$, к которому применили $\tanh$.  
$o$ называют "output gate" или вентилем выхода. Он отвечает за то, что добавится в новую кратковременную память из новой долгосрочной памяти.

Во время работы ячейки 8 афинных преобразований (4 для $x$ и 4 для $h$) применяются к одним и тем же векторам. Для эффективности вычислений можно 4 афинных преобразования над вектором $x$ (аналогично и для вектора $h$) произвести одновременно с помощью одного матричного произведения вместо трех (подумайте, почему так будет быстрее!). Для этого:
- Матрицы $W_{i?}$ объединяют в одну $W_{ih} \in \mathbb{R}^{4d_{hidden} \times d_{in}}$;
- Матрицы $W_{h?}$ объединяют в одну $W_{hh} \in \mathbb{R}^{4d_{hidden} \times d_{hidden}}$;
- Векторы $b_{i?}$ объединяют в один $b_{ih} \in \mathbb{R}^{4d_{hidden}}$;
- Векторы $b_{h?}$ объединяют в один $b_{hh} \in \mathbb{R}^{4d_{hidden}}$;
- Во время работы ячейки к входу $x$ и внутреннему состоянию $h$ применяются сразу все афинные преобразования с помощью матричного произведения $q$ = $W_{ih} x + b_{ih} + W_{hh} h + b_{hh}$.
- После этого вектор $q \in \mathbb{R}^{4 d_{hidden}}$ разрезается на четыре части $i, f, g, o$. Все оставшиеся вычисления выполняются по формулам.

В реализации `torch.nn.LSTMCell` для эффективности вычислений применяется именно этот трюк.

Подробнее про LSTM сети можно [почитать тут](https://colah.github.io/posts/2015-08-Understanding-LSTMs/).

---

На практике обычно не используют RNN-cell в силу ее простоты. GRU-cell и LSTM-cell обычно показывают схожее качество на реальных задачах.

**Резюме главы**
1. Есть несколько видов ячеек RNN: RNN-cell, GRU-cell, LSTM-cell.
2. На практике используют GRU и LSTM сети.

[К оглавлению](#toc)

## RNN-ячейки в `torch` <a id='ssection_torch_rnn_cells'></a>

Давайте теперь подробнее посмотрим на работу с ячейками в коде. Будем использовать фреймворк для глубинного обучения `PyTorch`.

Импортируем библиотеку `torch`. Функция `shape(tensor)` предназначена для красивого распечатывания размера тензора.

In [1]:
import torch          # Импортируем библиотеку torch,
from torch import nn  #  а также модуль nn для работы с нейронными сетями
# Импорты библиотек были продублированы тут, чтобы можно было легко 
#  запускать куски этого ноутбука независимо друг от друга.


def shape(tensor):
    return ' x '.join(map(str, tensor.shape))

Зададим $d_{in}$, $d_{hidden}$, размер батча для обработки (в коде будем использовать обозначение `batch_size`) и длину входной последовательности ($T$, в коде будем использовать обозначение `seq_len`).

В нейронные сети данные обычно подаются батчами — небольшими кусками данных, в которых собрано сразу несколько последовательностей для обучения или инференса. Так делают для более высокой вычислительной эффективности нейронных сетей. У крупных моделей время обучения может составлять часы, дни, а иногда и недели, поэтому использование эффективных методов обучения моделей очень важно.

Обычно последовательности в батче обрабатываются независимо друг от друга (исключение тут составляют слои батч-нормализации). Поэтому если говорить про вычисления в рекуррентных ячейках, то просто у каждого вектора данных ($x, h$ и $c$) появляется новая размерность, отвечающая за номер последовательности в батче.

В RNN-ячейку мы передаем один очередной входной эмбеддинг для каждой из последовательностей в батче, а также соответствующие им внутренние состояния ($h$, $c$ в случае LSTM).

In [2]:
batch_size = 3
d_in = 7
d_hidden = 23
seq_len = 5

Сгенерируем случайные значения для входных векторов $x$ и для начальных внутренних состояний $h$ и $c$ ($c$ будет нужен только для LSTM). Заметим, что у тензоров `h0` и `c0` нет размерности `seq_len`, потому что внутреннее состояние ячейки "выделяется" сразу на всю последовательность целиком, меняется по мере прохождения $x_1, \dots, x_T$ через ячейку, и, следовательно, от длины не зависит.

In [3]:
x = torch.rand(seq_len, batch_size, d_in)
h0 = torch.rand(batch_size, d_hidden)
c0 = torch.rand(batch_size, d_hidden)

run_args = (d_in, d_hidden, x, h0, c0)

*Небольшое пояснение.* Мы считаем, что в `x[i, j]` находится $i$-ый эмбеддинг токена $j$-ой последовательности, в `h0[j]` лежит начальное скрытое состояние ячейки, соответствующее последовательности `x[:, j]`, а в `c0[j]` лежит соответствующая последовательности `x[:, j]` начальная долгосрочная память ячейки (в случае LSTM).

Для того, чтобы смотреть на работу различных RNN ячеек, напишем для этого удобную функцию `run_rnn_cell(...)`. Она распечатывает размеры тензоров входа, выхода, скрытых состояний и параметров ячейки.

В циклах `for t, x_t in enumerate(x)` мы постепенно пропускаем вход $x$ через ячейку. Заметьте, что на вход ячейка принимает батч эмбеддингов для текущего слова из каждой последовательности, то есть в ячейку мы не передаем все последовательности сразу целиком.

In [4]:
def run_rnn_cell(
    rnn_cell_type: str, d_in: int, d_hidden: int,
    x: torch.Tensor, h0: torch.Tensor, c0: torch.Tensor = None
):
    if hasattr(torch.nn, rnn_cell_type):
        cell_class = getattr(torch.nn, rnn_cell_type)
    else:
        cell_class = globals()[rnn_cell_type]

    cell = cell_class(d_in, d_hidden)

    # Цикл по параметрам ячейки
    print(f"Размеры параметров ячейки {type(cell).__name__}")
    for name, p in cell.named_parameters():
        print(f"  * Параметр {name} имеет размер {shape(p)}")

    if isinstance(cell, (nn.RNNCell, nn.GRUCell)):
        # ! RNNCell или GRUCell
        # Печатаем размеры входа и выхода ячейки
        print()
        print(f"Размер входа: {shape(x)}")
        print(f"Размер изначального скрытого состояния: {shape(h0)}")
        print()

        # Пропускаем x и h0 через ячейку
        h = h0
        for t, x_t in enumerate(x):
            h = cell(x_t, h)
            print(f"Размер скрытого состояния после шага {t + 1}: {shape(h)}")
    else:
        # ! LSTMCell или MyLSTMCell
        # Печатаем размеры входа и выхода ячейки
        print()
        print(f"Размер входа: {shape(x)}")
        print(f"Размер изначальной краткосрочной памяти: {shape(h0)}")
        print(f"Размер изначальной  долгосрочной памяти: {shape(c0)}")
        print()

        # Пропускаем x, h0 и c0 через ячейку
        h, c = h0, c0
        for t, x_t in enumerate(x):
            h, c = cell(x_t, (h, c))

            print(f"Размер краткосрочной памяти после шага {t + 1}: {shape(h)}")
            print(f"Размер  долгосрочной памяти после шага {t + 1}: {shape(c)}")
            print()

Посмотрим на работу `RNNCell`:

In [5]:
run_rnn_cell("RNNCell", *run_args)

Размеры параметров ячейки RNNCell
  * Параметр weight_ih имеет размер 23 x 7
  * Параметр weight_hh имеет размер 23 x 23
  * Параметр bias_ih имеет размер 23
  * Параметр bias_hh имеет размер 23

Размер входа: 5 x 3 x 7
Размер изначального скрытого состояния: 3 x 23

Размер скрытого состояния после шага 1: 3 x 23
Размер скрытого состояния после шага 2: 3 x 23
Размер скрытого состояния после шага 3: 3 x 23
Размер скрытого состояния после шага 4: 3 x 23
Размер скрытого состояния после шага 5: 3 x 23


Посмотрим на работу `GRUCell`:

In [6]:
run_rnn_cell("GRUCell", *run_args)

Размеры параметров ячейки GRUCell
  * Параметр weight_ih имеет размер 69 x 7
  * Параметр weight_hh имеет размер 69 x 23
  * Параметр bias_ih имеет размер 69
  * Параметр bias_hh имеет размер 69

Размер входа: 5 x 3 x 7
Размер изначального скрытого состояния: 3 x 23

Размер скрытого состояния после шага 1: 3 x 23
Размер скрытого состояния после шага 2: 3 x 23
Размер скрытого состояния после шага 3: 3 x 23
Размер скрытого состояния после шага 4: 3 x 23
Размер скрытого состояния после шага 5: 3 x 23


Посмотрим на работу `LSTMCell`:

In [7]:
run_rnn_cell("LSTMCell", *run_args)

Размеры параметров ячейки LSTMCell
  * Параметр weight_ih имеет размер 92 x 7
  * Параметр weight_hh имеет размер 92 x 23
  * Параметр bias_ih имеет размер 92
  * Параметр bias_hh имеет размер 92

Размер входа: 5 x 3 x 7
Размер изначальной краткосрочной памяти: 3 x 23
Размер изначальной  долгосрочной памяти: 3 x 23

Размер краткосрочной памяти после шага 1: 3 x 23
Размер  долгосрочной памяти после шага 1: 3 x 23

Размер краткосрочной памяти после шага 2: 3 x 23
Размер  долгосрочной памяти после шага 2: 3 x 23

Размер краткосрочной памяти после шага 3: 3 x 23
Размер  долгосрочной памяти после шага 3: 3 x 23

Размер краткосрочной памяти после шага 4: 3 x 23
Размер  долгосрочной памяти после шага 4: 3 x 23

Размер краткосрочной памяти после шага 5: 3 x 23
Размер  долгосрочной памяти после шага 5: 3 x 23



Как видим, в ячейках `GRUCell` и `LSTMCell` действительно матрицы афинных преобразований склеены, а также видим, что в `GRUCell` склеено 3 матрицы, а в `LSTMCell` — 4.

Работа ячеек `RNNCell`, `GRUCell` и `LSTMCell` реализована на C/C++ и Cuda для обеспечения высокой эффективности их работы. Ниже представлена возможная реализация `LSTMCell` на `Python` для более глубокого ознакомления читателя с возможностями фреймворка `torch` и работой `LSTMCell`.

Замечу, что в реализации в библиотеке `torch` используются дублирующие векторы сдвигов (bias) для совместимости с библиотекой cuDNN — CUDA Deep Neural Network (а точнее, для прямого вызова функций из cuDNN). Например, в $f = \sigma\left(W_{i f} x+b_{i f}+W_{h f} h+b_{h f}\right)$ понятно, что оба вектора $b_{if}$ и $b_{hf}$ не нужны и их можно заменить на один вектор $b_{f} \in \mathbb{R}^{d_{hidden}}$. Поэтому в предлагаемой реализации создается всего один вектор для сдвига вместо двух для большей эффективности вычислений.

В нашей реализации используется функция `torch.chunk`, которая равномерно разбивает тензор на заданное количество частей (в нашем случае — на 4) по заданной размерности. С помощью нее удобно как раз получить вектора $f, i, o$ и $g$ после афинных преобразований входа и скрытого состояния.

In [8]:
from typing import Tuple


class MyLSTMCell(nn.Module):
    def __init__(self, d_in: int, d_hidden: int):
        super().__init__()

        self.d_in = d_in
        self.d_hidden = d_hidden

        # Регистрируем self.weight_ih, self.weight_hh и self.bias_hh
        #  в качестве параметров нашего MyLSTMCell модуля. Это нужно для
        #  того, чтобы эти параметры обучались во время обучения.
        self.weight_ih = torch.nn.Parameter(
            data=torch.zeros(d_in, 4 * d_hidden)
        )

        self.weight_hh = torch.nn.Parameter(
            data=torch.zeros(d_hidden, 4 * d_hidden)
        )

        self.bias_h = torch.nn.Parameter(
            data=torch.zeros(4 * d_hidden)
        )

        # Можно создать и два вектора для сдвига, если очень хочется:
        # self.bias_ih = torch.nn.Parameter(
        #     data=torch.zeros(4 * d_hidden)
        # )
        # self.bias_hh = torch.nn.Parameter(
        #     data=torch.zeros(4 * d_hidden)
        # )

    def forward(self, x: torch.Tensor, h0_c0: Tuple[torch.Tensor, torch.Tensor]):
        h0, c0 = h0_c0

        Wx = torch.matmul(x, self.weight_ih)
        Uh = torch.matmul(h0, self.weight_hh)

        fiog = Wx + Uh + self.bias_h  # batch_size x 4*d_hidden
        f, i, o, g = torch.chunk(fiog, 4, dim=1)
        f, i, o = map(torch.sigmoid, [f, i, o])
        g = torch.tanh(g)

        c1 = f * c0 + i * g
        h1 = o * torch.tanh(c1)

        return h1, c1

In [9]:
run_rnn_cell("MyLSTMCell", *run_args)

Размеры параметров ячейки MyLSTMCell
  * Параметр weight_ih имеет размер 7 x 92
  * Параметр weight_hh имеет размер 23 x 92
  * Параметр bias_h имеет размер 92

Размер входа: 5 x 3 x 7
Размер изначальной краткосрочной памяти: 3 x 23
Размер изначальной  долгосрочной памяти: 3 x 23

Размер краткосрочной памяти после шага 1: 3 x 23
Размер  долгосрочной памяти после шага 1: 3 x 23

Размер краткосрочной памяти после шага 2: 3 x 23
Размер  долгосрочной памяти после шага 2: 3 x 23

Размер краткосрочной памяти после шага 3: 3 x 23
Размер  долгосрочной памяти после шага 3: 3 x 23

Размер краткосрочной памяти после шага 4: 3 x 23
Размер  долгосрочной памяти после шага 4: 3 x 23

Размер краткосрочной памяти после шага 5: 3 x 23
Размер  долгосрочной памяти после шага 5: 3 x 23



**Резюме главы**
1. Изучили работу RNN-ячеек в `torch`;
2. Рассмотрели реализацию LSTM-ячейки на `torch`.

[К оглавлению](#toc)

## RNN слои в `torch` <a id='ssection_torch_rnn'></a>

В библиотеке `torch` помимио RNN-ячеек реализованы также и полноценные RNN-слои:
- `torch.nn.RNN`
- `torch.nn.GRU`
- `torch.nn.LSTM`

Отличаются они между собой только тем, какие внутри них используются RNN-ячейки.

При создании каждого из этих слоев можно задать следующие параметры:
1. `input_size` — размерность эмбеддингов входа ($d_{in}$);
2. `hidden_size` — размерность внутреннего состояния ($d_{hidden}$);
3. `num_layers` — количество слоев в сети;
4. `bidirectional` — является ли сеть двунаправленной или нет;
5. `bias` — использовать ли сдвиг при афинных преобразованиях;
6. `batch_first` — можно задавать, в какой последовательности будут идти размерности входного и выходного тензоров. В случае `batch_first = False` предполагается, что первая размерность $x$ отвечает за длину последовательности, вторая размерность — за отдельные последовательности в батче, третья — за размерность эмбеддингов. В случае `batch_first = True` предполагается, что первая и вторая размерности $x$ поменяны местами, то есть, что первая размерность $x$ отвечает за отдельные последовательности в батче, вторая размерность — за длину последовательности, третья — за размерность эмбеддингов. Значение по умолчанию `batch_first = False` (на практике также обычно используют этот вариант).

Также у RNN слоев есть еще и другие параметры. Тут перечислены пока что самые важные параметры для вашего понимания.

Во время работы RNN слой принимает на вход сразу все `batch_size` последовательностей целиком, прокручивает их через ячейки и возвращает следующее:
1. Результат `output` работы слоя в виде тензора размера $T \times \text{batch_size} \times D \cdot d_{hidden}$, где $D = 1$, если сеть однонаправленная, и $D = 2$, если сеть двунаправленная.
2. Внутренние состояния `h_n` для каждой последовательности в батче и каждого отдельного слоя. `h_n` имеет размеры $D \cdot \text{num_layers} \times \text{batch_size} \times H_{hidden}$.

Таким образом на выходе из сети имеем тензор `output`, совпадающий по описанию с последовательностью $y_1, \dots, y_T$ из [главы про функционирование RNN](#ssection_rnn_functioning).

Работа слоев `RNN`, `GRU` и `LSTM` реализована на C/C++ и Cuda/cuDNN для обеспечения высокой эффективности их работы. Из-за этого реализация RNN слоев на Python с использованием RNN-ячеек будет работать медленнее, поэтому RNN-ячейки следует использовать только в случае надобности и когда вы понимаете, что делаете.

**Резюме главы**
1. В `torch` есть реализованные RNN слои, полностью реализующие рассмотренный функционал рекуррентных нейронных сетей.
2. Не стоит переизобретать RNN слои без надобности.

[К оглавлению](#toc)

## Пример использования RNN слоев в `torch` <a id='ssection_torch_rnn_example'></a>

Давайте теперь подробнее посмотрим на работу с RNN в коде.

In [1]:
import torch          # Импортируем библиотеку torch,
from torch import nn  #  а также модуль nn для работы с нейронными сетями
# Импорты библиотек были продублированы тут, чтобы можно было легко 
#  запускать куски этого ноутбука независимо друг от друга.


def shape(tensor):
    return ' x '.join(map(str, tensor.shape))

Зададим $d_{in}$, $d_{hidden}$, размер батча для обработки (в коде будем использовать обозначение `batch_size`), длину входной последовательности ($T$, в коде будем использовать обозначение `seq_len`), а также параметры RNN: количество слоев `num_layers` и индикатор двунаправленности сети `bidirectional`.

В RNN слой мы передаем сразу все входные эмбеддинги для каждой из последовательностей в батче, а также соответствующие им внутренние состояния ($h$, $c$ в случае LSTM).

In [2]:
batch_size = 3
d_in = 7
d_hidden = 23
seq_len = 5

num_layers = 2
bidirectional = False

Сгенерируем случайные значения для входных векторов $x$ и для начальных внутренних состояний $h$ и $c$ ($c$ будет нужен только для LSTM). Заметим, что у тензоров `h0` и `c0` нет размерности `seq_len`, потому что внутреннее состояние ячейки "выделяется" сразу на всю последовательность целиком, меняется по мере прохождения $x_1, \dots, x_T$ через ячейку, и, следовательно, от длины не зависит.

In [3]:
D = 2 if bidirectional else 1

x = torch.rand(seq_len, batch_size, d_in)
h0 = torch.rand(D * num_layers, batch_size, d_hidden)
c0 = torch.rand(D * num_layers, batch_size, d_hidden)

run_args = (d_in, d_hidden, num_layers, bidirectional, x, h0, c0)

*Небольшое пояснение.* Мы считаем, что в `x[i, j]` находится $i$-ый эмбеддинг токена $j$-ой последовательности, в `h0[l, j]` лежит начальное скрытое состояние слоя $l$, соответствующее последовательности `x[:, j]`, а в `c0[l, j]` лежит соответствующая последовательности `x[:, j]` начальная долгосрочная память слоя $l$ (в случае LSTM).

Для того, чтобы смотреть на работу различных RNN слоев, опишем для этого удобную функцию `run_rnn(...)`. Она распечатывает размеры тензоров входа, выхода, скрытых состояний и параметров слоя.

In [4]:
def run_rnn(
    rnn_type: str, d_in: int, d_hidden: int,
    num_layers: int, bidirectional: bool,
    x: torch.Tensor, h0: torch.Tensor, c0: torch.Tensor = None
):
    rnn_class = getattr(torch.nn, rnn_type)
    rnn = rnn_class(d_in, d_hidden, num_layers, bidirectional=bidirectional)

    # Цикл по параметрам RNN
    print(f"Размеры параметров слоя {type(rnn).__name__}"
          f" ({num_layers} слоя, двунаправленная: {bidirectional})")
    for name, p in rnn.named_parameters():
        print(f"  * Параметр {name} имеет размер {shape(p)}")

    if isinstance(rnn, torch.nn.LSTM):
        # ! LSTM
        # Печатаем размеры входа и выхода ячейки
        print()
        print(f"Размер входа: {shape(x)}")
        print(f"Размер изначальной краткосрочной памяти: {shape(h0)}")
        print(f"Размер изначальной  долгосрочной памяти: {shape(c0)}")
        print()

        # Пропускаем x, h0 и c0 через ячейку
        output, (hn, cn) = rnn(x, (h0, c0))
        print(f"Размер выхода: {shape(output)}")
        print(f"Размер конечной краткосрочной памяти: {shape(hn)}")
        print(f"Размер конечной  долгосрочной памяти: {shape(cn)}")
    else:
        # ! RNN или GRU
        # Печатаем размеры входа и выхода ячейки
        print()
        print(f"Размер входа: {shape(x)}")
        print(f"Размер изначального скрытого состояния: {shape(h0)}")
        print()

        # Пропускаем x и h0 через ячейку
        output, hn = rnn(x, h0)
        print(f"Размер выхода: {shape(output)}")
        print(f"Размер конечного скрытого состояния: {shape(hn)}")

Посмотрим на работу `RNN`:

In [5]:
run_rnn("RNN", *run_args)

Размеры параметров слоя RNN (2 слоя, двунаправленная: False)
  * Параметр weight_ih_l0 имеет размер 23 x 7
  * Параметр weight_hh_l0 имеет размер 23 x 23
  * Параметр bias_ih_l0 имеет размер 23
  * Параметр bias_hh_l0 имеет размер 23
  * Параметр weight_ih_l1 имеет размер 23 x 23
  * Параметр weight_hh_l1 имеет размер 23 x 23
  * Параметр bias_ih_l1 имеет размер 23
  * Параметр bias_hh_l1 имеет размер 23

Размер входа: 5 x 3 x 7
Размер изначального скрытого состояния: 2 x 3 x 23

Размер выхода: 5 x 3 x 23
Размер конечного скрытого состояния: 2 x 3 x 23


Посмотрим на работу `GRU`:

In [6]:
run_rnn("GRU", *run_args)

Размеры параметров слоя GRU (2 слоя, двунаправленная: False)
  * Параметр weight_ih_l0 имеет размер 69 x 7
  * Параметр weight_hh_l0 имеет размер 69 x 23
  * Параметр bias_ih_l0 имеет размер 69
  * Параметр bias_hh_l0 имеет размер 69
  * Параметр weight_ih_l1 имеет размер 69 x 23
  * Параметр weight_hh_l1 имеет размер 69 x 23
  * Параметр bias_ih_l1 имеет размер 69
  * Параметр bias_hh_l1 имеет размер 69

Размер входа: 5 x 3 x 7
Размер изначального скрытого состояния: 2 x 3 x 23

Размер выхода: 5 x 3 x 23
Размер конечного скрытого состояния: 2 x 3 x 23


Посмотрим на работу `LSTM`:

In [7]:
run_rnn("LSTM", *run_args)

Размеры параметров слоя LSTM (2 слоя, двунаправленная: False)
  * Параметр weight_ih_l0 имеет размер 92 x 7
  * Параметр weight_hh_l0 имеет размер 92 x 23
  * Параметр bias_ih_l0 имеет размер 92
  * Параметр bias_hh_l0 имеет размер 92
  * Параметр weight_ih_l1 имеет размер 92 x 23
  * Параметр weight_hh_l1 имеет размер 92 x 23
  * Параметр bias_ih_l1 имеет размер 92
  * Параметр bias_hh_l1 имеет размер 92

Размер входа: 5 x 3 x 7
Размер изначальной краткосрочной памяти: 2 x 3 x 23
Размер изначальной  долгосрочной памяти: 2 x 3 x 23

Размер выхода: 5 x 3 x 23
Размер конечной краткосрочной памяти: 2 x 3 x 23
Размер конечной  долгосрочной памяти: 2 x 3 x 23


Как видим, в двух-слойных RNN действительно содержится по 2 RNN-ячейки.

Попробуем теперь добавить еще двунаправленность для RNN:

In [8]:
bidirectional = True

In [9]:
D = 2 if bidirectional else 1

x = torch.rand(seq_len, batch_size, d_in)
h0 = torch.rand(D * num_layers, batch_size, d_hidden)
c0 = torch.rand(D * num_layers, batch_size, d_hidden)

run_args = (d_in, d_hidden, num_layers, bidirectional, x, h0, c0)

Посмотрим на работу `RNN`:

In [10]:
run_rnn("RNN", *run_args)

Размеры параметров слоя RNN (2 слоя, двунаправленная: True)
  * Параметр weight_ih_l0 имеет размер 23 x 7
  * Параметр weight_hh_l0 имеет размер 23 x 23
  * Параметр bias_ih_l0 имеет размер 23
  * Параметр bias_hh_l0 имеет размер 23
  * Параметр weight_ih_l0_reverse имеет размер 23 x 7
  * Параметр weight_hh_l0_reverse имеет размер 23 x 23
  * Параметр bias_ih_l0_reverse имеет размер 23
  * Параметр bias_hh_l0_reverse имеет размер 23
  * Параметр weight_ih_l1 имеет размер 23 x 46
  * Параметр weight_hh_l1 имеет размер 23 x 23
  * Параметр bias_ih_l1 имеет размер 23
  * Параметр bias_hh_l1 имеет размер 23
  * Параметр weight_ih_l1_reverse имеет размер 23 x 46
  * Параметр weight_hh_l1_reverse имеет размер 23 x 23
  * Параметр bias_ih_l1_reverse имеет размер 23
  * Параметр bias_hh_l1_reverse имеет размер 23

Размер входа: 5 x 3 x 7
Размер изначального скрытого состояния: 4 x 3 x 23

Размер выхода: 5 x 3 x 46
Размер конечного скрытого состояния: 4 x 3 x 23


Посмотрим на работу `GRU`:

In [11]:
run_rnn("GRU", *run_args)

Размеры параметров слоя GRU (2 слоя, двунаправленная: True)
  * Параметр weight_ih_l0 имеет размер 69 x 7
  * Параметр weight_hh_l0 имеет размер 69 x 23
  * Параметр bias_ih_l0 имеет размер 69
  * Параметр bias_hh_l0 имеет размер 69
  * Параметр weight_ih_l0_reverse имеет размер 69 x 7
  * Параметр weight_hh_l0_reverse имеет размер 69 x 23
  * Параметр bias_ih_l0_reverse имеет размер 69
  * Параметр bias_hh_l0_reverse имеет размер 69
  * Параметр weight_ih_l1 имеет размер 69 x 46
  * Параметр weight_hh_l1 имеет размер 69 x 23
  * Параметр bias_ih_l1 имеет размер 69
  * Параметр bias_hh_l1 имеет размер 69
  * Параметр weight_ih_l1_reverse имеет размер 69 x 46
  * Параметр weight_hh_l1_reverse имеет размер 69 x 23
  * Параметр bias_ih_l1_reverse имеет размер 69
  * Параметр bias_hh_l1_reverse имеет размер 69

Размер входа: 5 x 3 x 7
Размер изначального скрытого состояния: 4 x 3 x 23

Размер выхода: 5 x 3 x 46
Размер конечного скрытого состояния: 4 x 3 x 23


Посмотрим на работу `LSTM`:

In [12]:
run_rnn("LSTM", *run_args)

Размеры параметров слоя LSTM (2 слоя, двунаправленная: True)
  * Параметр weight_ih_l0 имеет размер 92 x 7
  * Параметр weight_hh_l0 имеет размер 92 x 23
  * Параметр bias_ih_l0 имеет размер 92
  * Параметр bias_hh_l0 имеет размер 92
  * Параметр weight_ih_l0_reverse имеет размер 92 x 7
  * Параметр weight_hh_l0_reverse имеет размер 92 x 23
  * Параметр bias_ih_l0_reverse имеет размер 92
  * Параметр bias_hh_l0_reverse имеет размер 92
  * Параметр weight_ih_l1 имеет размер 92 x 46
  * Параметр weight_hh_l1 имеет размер 92 x 23
  * Параметр bias_ih_l1 имеет размер 92
  * Параметр bias_hh_l1 имеет размер 92
  * Параметр weight_ih_l1_reverse имеет размер 92 x 46
  * Параметр weight_hh_l1_reverse имеет размер 92 x 23
  * Параметр bias_ih_l1_reverse имеет размер 92
  * Параметр bias_hh_l1_reverse имеет размер 92

Размер входа: 5 x 3 x 7
Размер изначальной краткосрочной памяти: 4 x 3 x 23
Размер изначальной  долгосрочной памяти: 4 x 3 x 23

Размер выхода: 5 x 3 x 46
Размер конечной краткосро

Можно заметить, что в сетях появились параметры с суффиксом `_reverse` — это как раз ячейки, которые отвечают за чтение входа справа-налево. Также интересно тут поразглядывать размерности параметров промежуточных слоев. Например, в LSTM параметр `weight_ih_l1` имеет размерности $4 \cdot d_{hidden} \times 2 \cdot d_{hidden}$, потому что на вход он получает с предыдущего слоя (с индексом 0) комбинированный вектор $y_t^0$, полученный после конкатенации векторов $\overrightarrow{y_t}^{0}$ и $\overleftarrow{y_t}^{0}$.

**По итогу хотелось бы еще раз повторить:**
1. На практике стоит использовать `LSTM` или `GRU` сети с 1-3 слоями.
2. Двунаправленность обычно позволяет улучшить качество решения задачи, но из-за нее увеличивается количество обучаемых параметров более, чем вдвое. К тому же иногда нет смысла делать RNN двунаправленной.
3. RNN-<u>ячейки</u> не стоит использовать без необходимости.

[К оглавлению](#toc)

## Упаковка последовательностей разной длины <a id='ssection_sequences_packing'></a>
Возможно, вы, пока читали этот сопроводительный ноутбук, задались вопросом, как мы можем запихнуть в один батч несколько последовательностей разной длины?
Напомню, что необходимость класть несколько последовательностей в один батч у нас появилась во время экспериментов над RNN слоями.

### Упаковка последовательности <a id='sssection_sequences_packing_filtration'></a>
В батче для *RNN-ячеек* мы хранили лишь по одному эмбеддингу из каждой последовательности, поэтому при работе с ними у нас была только необходимость положить в батч эмбеддинги для текущего слова из каждой последовательности, а с этим проблем не возникает. Единственное тонкое место тут, а что делать, если в каких-то последовательностях закончились слова, а в каких-то — нет? Тогда мы можем сформировать батч только из эмбеддингов тех последовательностей, в которых слова еще не кончились, и подать их на вход сети. Тут надо аккуратно отфильтровать еще и внутренние состояния сети для подачи их в ячейку.

Обратимся к примеру, чтобы стало понятнее о чем идет речь. Для начала импортируем `torch`:

In [1]:
import torch          # Импортируем библиотеку torch,
from torch import nn  #  а также модуль nn для работы с нейронными сетями
# Импорты библиотек были продублированы тут, чтобы можно было легко 
#  запускать куски этого ноутбука независимо друг от друга.

def shape(tensor):
    return ' x '.join(map(str, tensor.shape))

Создадим список предложений, которые будем сейчас пропускать через ячейку RNN:

In [2]:
texts = [
    ["hey"],
    ["deadline", "is", "tomorrow"],
    ["so", "have", "you", "started", "doing", "anything"],
    ["what"],
    ["oh", "sure"]
]

Создадим по тексту словарь и отображение из слов в индексы. В целом, словарь нам не очень нужен, но с помощью него проще создавать отображение слов в индексы.

In [3]:
vocabulary = {
    word
    for text in texts
    for word in text
}

vocabulary

{'anything',
 'deadline',
 'doing',
 'have',
 'hey',
 'is',
 'oh',
 'so',
 'started',
 'sure',
 'tomorrow',
 'what',
 'you'}

In [4]:
word2ind = {
    word: index
    for index, word
    in enumerate(vocabulary)
}

word2ind

{'doing': 0,
 'is': 1,
 'have': 2,
 'deadline': 3,
 'tomorrow': 4,
 'you': 5,
 'what': 6,
 'started': 7,
 'anything': 8,
 'oh': 9,
 'hey': 10,
 'sure': 11,
 'so': 12}

Переводим предложения в индексы:

In [5]:
texts_idxs = [
    [
        word2ind[word]
        for word in text
    ]
    for text in texts
]

texts_idxs

[[10], [3, 1, 4], [12, 2, 5, 7, 0, 8], [6], [9, 11]]

Создадим слой уже знакомых вам эмбеддингов:

In [6]:
d_in = 2      # Размерность эмбеддингов
d_hidden = 10 # Размерность скрытого состояния ячеек

embeddings = nn.Embedding(num_embeddings=len(vocabulary), embedding_dim=d_in)
embeddings

Embedding(13, 2)

Напишем цикл обработки предложений:

In [7]:
# Будем сюда сохранять итоговые состояния ячеек. Далее будет еще
#  несколько примеров реализации того, что написано в этой клетке.
#  В конце мы проверим, что все внутренние состояния в различных
#  реализациях совпадают.
hs = []

# Зададим зерно для генератора псевдослучайных чисел (ГПСЧ)
#  для воспроизводимости результатов, а именно чтобы всегда
#  RNN-ячейка и внутреннее состояние генерировались одинаковыми
#  от запуска к запуску.
torch.manual_seed(179)

# Создаем ячейки и инициализируем случайно внутренние состояния ячейки.
cell = nn.RNNCell(d_in, d_hidden)
h = torch.rand(len(texts_idxs), d_hidden)

length_longest_seq = max(map(len, texts_idxs))

# Цикл по длине последовательностей
for step in range(length_longest_seq):
    # Определим последовательности, в которых слова еще не кончились.
    mask_leave = [len(text_idxs) > step for text_idxs in texts_idxs]

    # Создадим батч из текущих слов
    batch = [
        text_idxs[step]
        for leave, text_idxs
        in zip(mask_leave, texts_idxs)
        if leave
    ]

    # Переводим батч в тензор
    batch = torch.LongTensor(batch)
    print(f"Текущий батч индексов слов: {batch}")

    # Получаем для него эмбеддинги
    embeds = embeddings(batch)
    print(f"Батч эмбеддингов для текущих слов имеет размер {shape(embeds)}")

    h_prev = h.clone()

    # Подаем RNN ячейке на вход эмбеддинги и
    #  соответствующие им внутренние состояния ячейки
    h[mask_leave] = cell(embeds, h[mask_leave])
    print(f"Внутренние состояния для последовательностей,\n"
          f"  которые пропускались через RNN, имеют размер  {shape(h[mask_leave])}")
    print(f"Маска последовательностей, которые пока что еще пропускаются\n"
          f"  через RNN-ячейку:     {mask_leave}")
    print(f"Проверка того, для каких последовательностей изменились\n"
          f"  внутренние состояния: {(h != h_prev).any(dim=1).tolist()}")
    print()

hs.append(h.clone())

Текущий батч индексов слов: tensor([10,  3, 12,  6,  9])
Батч эмбеддингов для текущих слов имеет размер 5 x 2
Внутренние состояния для последовательностей,
  которые пропускались через RNN, имеют размер  5 x 10
Маска последовательностей, которые пока что еще пропускаются
  через RNN-ячейку:     [True, True, True, True, True]
Проверка того, для каких последовательностей изменились
  внутренние состояния: [True, True, True, True, True]

Текущий батч индексов слов: tensor([ 1,  2, 11])
Батч эмбеддингов для текущих слов имеет размер 3 x 2
Внутренние состояния для последовательностей,
  которые пропускались через RNN, имеют размер  3 x 10
Маска последовательностей, которые пока что еще пропускаются
  через RNN-ячейку:     [False, True, True, False, True]
Проверка того, для каких последовательностей изменились
  внутренние состояния: [False, True, True, False, True]

Текущий батч индексов слов: tensor([4, 5])
Батч эмбеддингов для текущих слов имеет размер 2 x 2
Внутренние состояния для после

В выводе ячейки выше можно пронаблюдать, как постепенно предложения пропускаются через `RNNCell`. Чтобы упростить понимание того, что происходит, можно обратиться к иллюстрации упаковки последовательностей:

![Packed or padded sequences](images/packed-padded-sequences.jpeg "Packed or padded sequences")
На рисунке последовательности обозначены черным. Последовательности начинаются слева и заканчиваются справа, а сверху вниз приведены несколько последовательностей. Цветами выделены батчи. Справа на рисунке изображены размеры батчей на каждом шаге.

#### Несколько трюков <a id='ssssection_sequences_packing_filtration_tricks'></a>
При создании батча по маске можно было бы использовать функцию `compress` из стандартной библиотеки `itertools`. Данная функция осуществляет маскирование для списков.
```python
batch = [
    text_idxs[step]
    for leave, text_idxs
    in zip(mask_leave, texts_idxs)
    if leave
]
```

$\rightarrow$

```python
batch = [
    text_idxs[step] for text_idxs
    in compress(texts_idxs, mask_leave)
]
```

Пример работы этой функции:

In [8]:
from itertools import compress


list(compress([1, 2, 3, 4], [True, False, False, True]))

[1, 4]

Также сам способ образования батчей можно было бы возложить на стандартную функцию `zip`. С помощью этой функции можно транспонировать вложенные объекты в питоне. Чтобы стало понятно, о чем идет речь, приведем следующий пример:

In [9]:
a = [
    [1, 2, 3],
    [4, 5, 6]
]

a_transposed = list(zip(*a))

for row in a_transposed:
    print(row)

(1, 4)
(2, 5)
(3, 6)


Чтобы понять, почему `zip(*a)` работает так, как работает, немного подумайте об этом сами ;)

<details>
<summary><font color='blue'>Нажмите сюда для объяснения</font></summary>

Мы по сути передаем в функцию `zip` все строки нашей "матрицы". Функция `zip` выдает нам склеенные первые элементы из каждой строки, затем выдает склеенные вторые элементы из каждой строки и так далее до конца каждой из строк. Поэтому на выходе мы имеем сначала первый столбец матрицы, затем второй и т.д. до последнего столбца. А математически мы тут получили как раз транспонирование матрицы.
</details>

А мы как раз и транспонируем список `texts_idxs`. Проблема с ним в том, что в нем строки разной длины (мы как раз сейчас здесь из-за этого и страдаем). Функция `zip` транспонирует только самые длинные "столбцы" в нашем списке, потому что `zip` выдает столбцы до тех пор, пока не кончатся элементы хотя бы в одной последовательности. Для того, чтобы побороться с этим, будем использовать функцию `zip_longest` из стандартной библиотеки `itertools`. Эта функция будет выдавать строки, пока хотя бы в одной строке еще есть элементы. Строки, в которых элементы уже кончились, дополняются `None` до длины самой длинной строки.

Посмотрим на работу этой функции на примере:

In [10]:
from itertools import zip_longest


for col in zip_longest(
    [1, 2, 3],
    "ab",
    [9, 8, 7, 6, 5]
):
    print(col)

(1, 'a', 9)
(2, 'b', 8)
(3, None, 7)
(None, None, 6)
(None, None, 5)


С учетом замечаний выше перепишем цикл обработки предложений:

In [11]:
torch.manual_seed(179)

# Создаем ячейки и инициализируем случайно внутренние состояния ячейки.
cell = nn.RNNCell(d_in, d_hidden)
h = torch.rand(len(texts_idxs), d_hidden)

length_longest_seq = max(map(len, texts_idxs))

# Цикл по длине последовательностей
for batch_long in zip_longest(*texts_idxs):
    print(f"Неочищенный батч: {batch_long}")

    # Определим последовательности, в которых слова еще не кончились.
    mask_leave = [ind is not None for ind in batch_long]

    # Почистим батч от None
    batch = list(compress(batch_long, mask_leave))

    # Переводим батч в тензор
    batch = torch.LongTensor(batch)
    print(f"Текущий батч индексов слов: {batch}")

    # Получаем для него эмбеддинги
    embeds = embeddings(batch)
    print(f"Батч эмбеддингов для текущих слов имеет размер {shape(embeds)}")

    h_prev = h.clone()

    # Подаем RNN ячейке на вход эмбеддинги и
    #  соответствующие им внутренние состояния ячейки
    h[mask_leave] = cell(embeds, h[mask_leave])
    print(f"Внутренние состояния для последовательностей,\n"
          f"  которые пропускались через RNN, имеют размер  {shape(h[mask_leave])}")
    print(f"Маска последовательностей, которые пока что еще пропускаются\n"
          f"  через RNN-ячейку:     {mask_leave}")
    print(f"Проверка того, для каких последовательностей изменились\n"
          f"  внутренние состояния: {(h != h_prev).any(dim=1).tolist()}")
    print()

hs.append(h.clone())

Неочищенный батч: (10, 3, 12, 6, 9)
Текущий батч индексов слов: tensor([10,  3, 12,  6,  9])
Батч эмбеддингов для текущих слов имеет размер 5 x 2
Внутренние состояния для последовательностей,
  которые пропускались через RNN, имеют размер  5 x 10
Маска последовательностей, которые пока что еще пропускаются
  через RNN-ячейку:     [True, True, True, True, True]
Проверка того, для каких последовательностей изменились
  внутренние состояния: [True, True, True, True, True]

Неочищенный батч: (None, 1, 2, None, 11)
Текущий батч индексов слов: tensor([ 1,  2, 11])
Батч эмбеддингов для текущих слов имеет размер 3 x 2
Внутренние состояния для последовательностей,
  которые пропускались через RNN, имеют размер  3 x 10
Маска последовательностей, которые пока что еще пропускаются
  через RNN-ячейку:     [False, True, True, False, True]
Проверка того, для каких последовательностей изменились
  внутренние состояния: [False, True, True, False, True]

Неочищенный батч: (None, 4, 5, None, None)
Текущи

Теперь код стал чище.

Небольшая проблема такой реализации в том, что на каждой итерации цикла мы просматриваем все предложения и выделяем из них достаточно длинные для текущего шага. Можно сэкономить на этом и на каждой итерации оставлять из еще не выкинутых последовательностей только нужные последовательности.

Для этого отсортируем последовательности по длине по убыванию. Тогда на каждом шаге в RNN-ячейку будут подаваться только несколько первых последовательностей. Значит, нам достаточно поддерживать количество еще незакончившихся предложений на каждом шаге. Такое решение будет работать быстрее (анализ сложности приведен в конце главы).

Отсортируем тексты в порядке убывания их длины. Для того, чтобы потом можно было восстановить исходный порядок внутренних состояний, сделаем `argsort` для длин последовательностей:

In [12]:
texts_idxs_argsorted = sorted(
    range(len(texts_idxs)),
    key=lambda i: len(texts_idxs[i]),
    reverse=True
)

texts_idxs_argsorted

[2, 1, 4, 0, 3]

На `numpy` такое можно реализовать проще с помощью функции `numpy.argsort`:

In [13]:
import numpy as np


texts_idxs_argsorted = np.argsort(list(map(lambda x: -len(x), texts_idxs))).tolist()
texts_idxs_argsorted

[2, 1, 4, 0, 3]

Переупорядочим предложения в порядке убывания длины:

In [14]:
texts_idxs_sorted = [texts_idxs[i] for i in texts_idxs_argsorted]
texts_idxs_sorted

[[12, 2, 5, 7, 0, 8], [3, 1, 4], [9, 11], [10], [6]]

Составим обратное отображения для "отмены" сортировки для возвращения к первоначальному порядку предложений:

In [15]:
texts_idxs_argsort_return = np.argsort(texts_idxs_argsorted).tolist()
texts_idxs_argsort_return

[3, 1, 0, 4, 2]

Теперь напишем ускоренный цикл прохода по предложениям:

In [16]:
torch.manual_seed(179)

# Создаем ячейки и инициализируем случайно внутренние состояния ячейки.
cell = nn.RNNCell(d_in, d_hidden)
h = torch.rand(len(texts_idxs), d_hidden)[texts_idxs_argsorted]

num_active_seqs = len(texts_idxs)

# Цикл по длине последовательностей
for step in range(len(texts_idxs_sorted[0])):
    # Обновляем счетчик предложений, которые еще не кончились
    while len(texts_idxs_sorted[num_active_seqs - 1]) <= step:
        num_active_seqs -= 1

    # Составляем из них батч для слов
    batch = [
        text_idxs[step] for text_idxs
        in texts_idxs_sorted[:num_active_seqs]
    ]

    # Переводим батч в тензор
    batch = torch.LongTensor(batch)
    print(f"Текущий батч индексов слов: {batch}")

    # Получаем для него эмбеддинги
    embeds = embeddings(batch)
    print(f"Батч эмбеддингов для текущих слов имеет размер {shape(embeds)}")

    h_prev = h.clone()

    # Подаем RNN ячейке на вход эмбеддинги и
    #  соответствующие им внутренние состояния ячейки
    h[:num_active_seqs] = cell(embeds, h[:num_active_seqs])

    print(f"Внутренние состояния для последовательностей,\n"
          f"  которые пропускались через RNN, имеют размер  {shape(h[:num_active_seqs])}")
    print(f"Маска последовательностей, которые пока что еще пропускаются\n"
          f"  через RNN-ячейку:     {mask_leave}")
    print(f"Проверка того, для каких последовательностей изменились\n"
          f"  внутренние состояния: {(h != h_prev).any(dim=1).tolist()}")
    print()

hs.append(h.clone()[texts_idxs_argsort_return])

Текущий батч индексов слов: tensor([12,  3,  9, 10,  6])
Батч эмбеддингов для текущих слов имеет размер 5 x 2
Внутренние состояния для последовательностей,
  которые пропускались через RNN, имеют размер  5 x 10
Маска последовательностей, которые пока что еще пропускаются
  через RNN-ячейку:     [False, False, True, False, False]
Проверка того, для каких последовательностей изменились
  внутренние состояния: [True, True, True, True, True]

Текущий батч индексов слов: tensor([ 2,  1, 11])
Батч эмбеддингов для текущих слов имеет размер 3 x 2
Внутренние состояния для последовательностей,
  которые пропускались через RNN, имеют размер  3 x 10
Маска последовательностей, которые пока что еще пропускаются
  через RNN-ячейку:     [False, False, True, False, False]
Проверка того, для каких последовательностей изменились
  внутренние состояния: [True, True, True, False, False]

Текущий батч индексов слов: tensor([5, 4])
Батч эмбеддингов для текущих слов имеет размер 2 x 2
Внутренние состояния для

Проверим, что все реализации вернули в итоге одинаковые внутренние состояния ячеек:

In [17]:
all(torch.allclose(h1, h2) for h1 in hs for h2 in hs)

True

Данный метод носит название *упаковки последовательностей*, постольку поскольку для его работы не требуется дополнительная память, и в реализации `torch` все последовательности сливаются в одну, тем самым упаковываясь.

### Уравнивание длин последовательностей (метод набивки) <a id='sssection_sequences_packing_padding'></a>

Другой подход основан на том, что в конец коротких последовательностей добавляются незначимые символы. Для этого вводят специальный символ `<PAD>` (padding — набивка, заполнение), который добавляется в конец последовательностей так, чтобы уравнять их длины.

Добавим этот токен в словарь и в отображение слов в индексы:

In [18]:
PAD_TOKEN = "<PAD>"
vocabulary.add(PAD_TOKEN)
word2ind[PAD_TOKEN] = len(word2ind)
word2ind[PAD_TOKEN]

13

Обновим слой эмбеддингов. Сохраним в нем старые эмбеддинги для воспроизводимости экспериментов (а точнее для того, чтобы показать, что подходы с упаковкой и с набивкой последовательностей дают разные результаты, но об этом позже):

In [19]:
embeddings_ = nn.Embedding(num_embeddings=len(vocabulary), embedding_dim=d_in)
embeddings_.weight.data[:len(vocabulary) - 1] = embeddings.weight.data
embeddings = embeddings_
embeddings

Embedding(14, 2)

Заполним все короткие последовательности в конце "набивкой":

In [20]:
length_longest_seq = max(map(len, texts_idxs))

texts_idxs_padded = [
    text + [word2ind[PAD_TOKEN]] * (length_longest_seq - len(text))
    for text in texts_idxs
]
torch.LongTensor(texts_idxs_padded)

tensor([[10, 13, 13, 13, 13, 13],
        [ 3,  1,  4, 13, 13, 13],
        [12,  2,  5,  7,  0,  8],
        [ 6, 13, 13, 13, 13, 13],
        [ 9, 11, 13, 13, 13, 13]])

Теперь эти последовательности одинаковой длины и сложностей с подачей их в RNN-ячейку не возникает:

In [21]:
torch.manual_seed(179)

# Создаем ячейки и инициализируем случайно внутренние состояния ячейки.
cell = nn.RNNCell(d_in, d_hidden)
h = torch.rand(len(texts_idxs), d_hidden)

# Цикл по длине последовательностей
for batch in zip(*texts_idxs_padded):
    # Переводим батч в тензор
    batch = torch.LongTensor(batch)
    print(f"Текущий батч индексов слов: {batch}")

    # Получаем для него эмбеддинги
    embeds = embeddings(batch)
    print(f"Батч эмбеддингов для текущих слов имеет размер {shape(embeds)}")

    h_prev = h.clone()

    # Подаем RNN ячейке на вход эмбеддинги и
    #  соответствующие им внутренние состояния ячейки
    h = cell(embeds, h)

    print(f"Внутренние состояния для последовательностей,\n"
          f"  которые пропускались через RNN, имеют размер  {shape(h)}")
    print(f"Проверка того, для каких последовательностей изменились\n"
          f"  внутренние состояния: {(h != h_prev).any(dim=1).tolist()}")
    print()

hs.append(h.clone())

Текущий батч индексов слов: tensor([10,  3, 12,  6,  9])
Батч эмбеддингов для текущих слов имеет размер 5 x 2
Внутренние состояния для последовательностей,
  которые пропускались через RNN, имеют размер  5 x 10
Проверка того, для каких последовательностей изменились
  внутренние состояния: [True, True, True, True, True]

Текущий батч индексов слов: tensor([13,  1,  2, 13, 11])
Батч эмбеддингов для текущих слов имеет размер 5 x 2
Внутренние состояния для последовательностей,
  которые пропускались через RNN, имеют размер  5 x 10
Проверка того, для каких последовательностей изменились
  внутренние состояния: [True, True, True, True, True]

Текущий батч индексов слов: tensor([13,  4,  5, 13, 13])
Батч эмбеддингов для текущих слов имеет размер 5 x 2
Внутренние состояния для последовательностей,
  которые пропускались через RNN, имеют размер  5 x 10
Проверка того, для каких последовательностей изменились
  внутренние состояния: [True, True, True, True, True]

Текущий батч индексов слов: ten

Как видим, внутреннее состояние ячейки после пропускания "набитых" предложений отличается от состояния ячейки после пропускания упакованных последовательностей:

In [22]:
(hs[0] == hs[-1]).all().item()

False

Это связано с тем, что эти способы не эквивалентны друг другу. При набивке предложений модели придется еще дополнительно выучить, что такен `<PAD>` незначимый. На практике с этим проблем обычно не возникает.

Также стоит отметить, что второй подход более прожорлив как по памяти, так и по времени исполнения. Первый метод работает за время от суммы длин всех последовательностей $O \left(\sum\limits_{i=1}^{n} l_i \right)$, где $l_i$ — длина $i$-го предложения, а $n$ — количество предложений, в то время как второй — за $O \left( n \cdot \max\limits_{1 \leqslant i \leqslant n} l_i \right)$.

В случае, когда в батч попало 1 очень длинное предложение и много коротких, первый метод будет работать гораздо быстрее второго.

Для того, чтобы метод набивки можно было использовать на практике, обычно перед разбивкой обучающией коллекции на батчи все предложения в обучающей выборке сортируются по длине, после чего батчи формируются из подряд идущих предложений. В таком случае будем иметь батчи со сбалансированными длинами предложений и второй метод будет работать в таком случае быстро. Надо отметить, что в таком случае метод упаковки тоже ускорится, потому что чем меньше итераций будет сделано на питоне и чем больше будет их сделано на том, что лежит под питоном в библиотеке `torch` (C/C++/Cuda) (а сбалансированность длин предложений в одном батче приведет именно к этому), тем быстрее будет работать программа.

### Реализация подходов упаковки и набивки в `torch` <a id='sssection_sequences_packing_torch'></a>

Оба подхода реализованы и в `torch`. С упакованными последовательностями умеют работать только RNN слои. Давайте познакомимся поближе с тем, как пользоваться этими методами в `torch`.

#### "Чистый" метод упаковки <a id='ssssection_sequences_packing_torch_packing'></a>

Для того, чтобы применить метод упаковки, используют функцию `torch.nn.utils.rnn.pack_sequence`:

In [23]:
from torch.nn.utils.rnn import pack_sequence, unpack_sequence


texts_idxs_packed = pack_sequence(
    list(map(torch.LongTensor, texts_idxs)),  # Переводим индексы в torch.Tensor
    enforce_sorted=False  # Говорим функции, что последовательности
                          #  не отсортированы по длине
)

texts_idxs_packed

PackedSequence(data=tensor([12,  3,  9, 10,  6,  2,  1, 11,  5,  4,  7,  0,  8]), batch_sizes=tensor([5, 3, 2, 1, 1, 1]), sorted_indices=tensor([2, 1, 4, 0, 3]), unsorted_indices=tensor([3, 1, 0, 4, 2]))

Как видим, эта функция вернула нам объект класса `PackedSequence`. В нем в поле `data` хранятся все переданные индексы, склеенные вместе и в таком порядке, в котором они будут нарезаться на батчи, в поле `batch_sizes` — размеры будущих батчей, в поле `sorted_indices` — argsort набора предложений по их длине по убыванию, в `unsorted_indices` — отображение, отменяющее сортировку. Все эти элементы должны быть уже вам знакомы из [примера выше](#sssection_sequences_packing_filtration).

К сожалению, запакованную последовательность нельзя напрямую подать в слой эмбеддингов. Как это обычно решается — смотрите дальше. Сейчас же для демонстрации инструментов библиотеки `torch` получим сначала эмбеддинги из последовательностей индексов, после чего уже запакуем их:

In [24]:
# Получаем эмбеддинги по индексам
embeds = [
    embeddings(torch.LongTensor(text_idxs))
    for text_idxs in texts_idxs
]

embeds_packed = pack_sequence(embeds, enforce_sorted=False)
embeds_packed

PackedSequence(data=tensor([[-0.3089, -0.2599],
        [-0.5259, -0.9778],
        [ 1.9426,  0.0565],
        [-1.0886, -0.1778],
        [-0.5785,  0.8667],
        [-1.6345, -1.3710],
        [-0.5695,  0.8870],
        [-1.3840,  0.5695],
        [ 0.8844, -0.4322],
        [-0.6290, -0.5983],
        [-1.8559, -0.1057],
        [ 0.9385,  1.1987],
        [ 0.9743, -0.7255]], grad_fn=<PackPaddedSequenceBackward0>), batch_sizes=tensor([5, 3, 2, 1, 1, 1]), sorted_indices=tensor([2, 1, 4, 0, 3]), unsorted_indices=tensor([3, 1, 0, 4, 2]))

RNN-ячейки не умеют обрабатывать запакованные последовательности, зато RNN слои умеют:

In [25]:
rnn = nn.RNN(d_in, d_hidden)
rnn_out, h = rnn(embeds_packed)
rnn_out

PackedSequence(data=tensor([[ 0.0110, -0.0157,  0.0856,  0.1760,  0.0864,  0.0103, -0.1458,  0.3176,
          0.1838,  0.4372],
        [ 0.0861,  0.2323,  0.0489,  0.1524,  0.1573, -0.2141, -0.1664,  0.4220,
         -0.0078,  0.4593],
        [ 0.1841, -0.4358,  0.6282,  0.3547, -0.3893,  0.4066, -0.4448,  0.3589,
          0.7114,  0.3939],
        [-0.0752,  0.0822, -0.1465,  0.1121,  0.2470, -0.0833, -0.0174,  0.2709,
         -0.0228,  0.4467],
        [-0.1646, -0.3052, -0.0366,  0.1627,  0.1050,  0.2699, -0.0173,  0.1102,
          0.3047,  0.4108],
        [ 0.0872,  0.4860, -0.2796, -0.0228,  0.2314, -0.4818,  0.0843,  0.4883,
         -0.3779,  0.6318],
        [-0.2345, -0.3999, -0.2641,  0.1596, -0.1575,  0.3391, -0.0839,  0.1112,
          0.3720,  0.5543],
        [ 0.1434,  0.1608,  0.0835, -0.0210,  0.2676, -0.1877,  0.6370,  0.4007,
         -0.0724,  0.7231],
        [-0.0477, -0.3715, -0.0469,  0.3614, -0.5024,  0.2629, -0.6510,  0.3041,
          0.5316,  0.4988],

Для того, чтобы распаковать результат, достаточно применить к нему функцию `nn.utils.rnn.unpack_sequence` — он преобразует `PackedSequence` обратно в список:

In [26]:
unpack_sequence(rnn_out)

[tensor([[-0.0752,  0.0822, -0.1465,  0.1121,  0.2470, -0.0833, -0.0174,  0.2709,
          -0.0228,  0.4467]], grad_fn=<IndexBackward0>),
 tensor([[ 0.0861,  0.2323,  0.0489,  0.1524,  0.1573, -0.2141, -0.1664,  0.4220,
          -0.0078,  0.4593],
         [-0.2345, -0.3999, -0.2641,  0.1596, -0.1575,  0.3391, -0.0839,  0.1112,
           0.3720,  0.5543],
         [ 0.3699,  0.2273,  0.3728, -0.0199,  0.0152, -0.3986,  0.1605,  0.4392,
          -0.1500,  0.6607]], grad_fn=<IndexBackward0>),
 tensor([[ 0.0110, -0.0157,  0.0856,  0.1760,  0.0864,  0.0103, -0.1458,  0.3176,
           0.1838,  0.4372],
         [ 0.0872,  0.4860, -0.2796, -0.0228,  0.2314, -0.4818,  0.0843,  0.4883,
          -0.3779,  0.6318],
         [-0.0477, -0.3715, -0.0469,  0.3614, -0.5024,  0.2629, -0.6510,  0.3041,
           0.5316,  0.4988],
         [ 0.3247,  0.2467,  0.0849, -0.0288,  0.1723, -0.4786,  0.5555,  0.3698,
          -0.4012,  0.7793],
         [-0.2974, -0.5972, -0.1718,  0.3397, -0.4330,  

#### "Чистый" метод набивки <a id='ssssection_sequences_packing_torch_padding'></a>

Для того, чтобы применить метод набивки, используют функцию `nn.utils.rnn.pad_sequence`:

In [27]:
from torch.nn.utils.rnn import pad_sequence, unpad_sequence


texts_idxs_padded = pad_sequence(
    map(torch.LongTensor, texts_idxs),
    padding_value=word2ind[PAD_TOKEN]
)
texts_idxs_padded

tensor([[10,  3, 12,  6,  9],
        [13,  1,  2, 13, 11],
        [13,  4,  5, 13, 13],
        [13, 13,  7, 13, 13],
        [13, 13,  0, 13, 13],
        [13, 13,  8, 13, 13]])

Аргумент `padding_value` в этой функции отвечает за то, каким значением набивать последовательности. Обратите внимание, что возвращенный тензор имеет размерность $\text{max_len} \times \text{batch_size}$.

Далее такой тензор можно сразу подавать в слой эмбеддингов, а затем и в RNN:

In [28]:
rnn = torch.nn.RNN(d_in, d_hidden)

embeds_padded = embeddings(torch.LongTensor(texts_idxs_padded))
print(f"Размер батча эмбеддингов: {shape(embeds_padded)}")

rnn_out, h = rnn(embeds_padded)
print(f"Размер выхода RNN: {shape(rnn_out)}")

Размер батча эмбеддингов: 6 x 5 x 2
Размер выхода RNN: 6 x 5 x 10


Для отбрасывания лишних элементов в `rnn_out` можно воспользоваться функцией `unpad_sequence` (необходимо вторым аргументом ей передать длины всех предложений):

In [29]:
unpad_sequence(rnn_out, list(map(len, texts)))

[tensor([[-0.4400,  0.4774, -0.2960, -0.6203, -0.0224,  0.4075,  0.4349, -0.2259,
           0.1720,  0.2626]], grad_fn=<IndexBackward0>),
 tensor([[-0.1496,  0.4860, -0.2871, -0.6479,  0.0280,  0.1644,  0.4466, -0.1621,
           0.2928,  0.2263],
         [-0.3377, -0.0591,  0.0341, -0.0281, -0.1591,  0.1987, -0.0344, -0.1461,
           0.2311,  0.0782],
         [-0.2465,  0.4341, -0.1841, -0.4571, -0.0351,  0.1364,  0.3790, -0.1842,
           0.3339,  0.2631]], grad_fn=<IndexBackward0>),
 tensor([[-0.3304,  0.2878, -0.1733, -0.4786, -0.0314,  0.1936,  0.3534, -0.0932,
           0.1621,  0.0706],
         [ 0.0689,  0.6979, -0.3995, -0.7337,  0.0294,  0.2808,  0.4190, -0.3470,
           0.5214,  0.6385],
         [ 0.4244, -0.2657,  0.1031,  0.1000, -0.0173, -0.4684, -0.1429, -0.1179,
           0.5351, -0.2645],
         [-0.6820,  0.6020, -0.2024, -0.7496, -0.1993,  0.4588,  0.5548, -0.3902,
           0.3146,  0.1845],
         [-0.1604, -0.5341,  0.3641,  0.6457, -0.2201, -

#### Совмещение обоих подходов <a id='ssssection_sequences_packing_torch_mixed'></a>

Для удобства в `torch` можно переходить от одного представления батча последовательностей к другому с помощью специальных функций `pack_padded_sequence` и `pad_packed_sequence`.

У нас уже есть подсчитанные представления `texts_idxs_padded` и `texts_idxs_packed`. Попробуем перевести их друг в друга.

`texts_idxs_padded` $\rightarrow$ `texts_idxs_packed`:

In [30]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


pack_padded_sequence(
    texts_idxs_padded,
    lengths=list(map(len, texts)),
    enforce_sorted=False
)

PackedSequence(data=tensor([12,  3,  9, 10,  6,  2,  1, 11,  5,  4,  7,  0,  8]), batch_sizes=tensor([5, 3, 2, 1, 1, 1]), sorted_indices=tensor([2, 1, 4, 0, 3]), unsorted_indices=tensor([3, 1, 0, 4, 2]))

In [31]:
texts_idxs_packed

PackedSequence(data=tensor([12,  3,  9, 10,  6,  2,  1, 11,  5,  4,  7,  0,  8]), batch_sizes=tensor([5, 3, 2, 1, 1, 1]), sorted_indices=tensor([2, 1, 4, 0, 3]), unsorted_indices=tensor([3, 1, 0, 4, 2]))

Заметим, что у функции `pack_padded_sequence` имеется обязательный аргумент `lengths`, с помощью которого определяются длины последовательностей в батче.

`texts_idxs_packed` $\rightarrow$ `texts_idxs_padded`:

In [32]:
pad_packed_sequence(
    texts_idxs_packed,
    padding_value=word2ind[PAD_TOKEN]
)

(tensor([[10,  3, 12,  6,  9],
         [13,  1,  2, 13, 11],
         [13,  4,  5, 13, 13],
         [13, 13,  7, 13, 13],
         [13, 13,  0, 13, 13],
         [13, 13,  8, 13, 13]]),
 tensor([1, 3, 6, 1, 2]))

In [33]:
texts_idxs_padded

tensor([[10,  3, 12,  6,  9],
        [13,  1,  2, 13, 11],
        [13,  4,  5, 13, 13],
        [13, 13,  7, 13, 13],
        [13, 13,  0, 13, 13],
        [13, 13,  8, 13, 13]])

Функция `pad_packed_sequence` возвращает помимо "набитых" последовательной еще и их длины.

Вспомним, что при упаковке последовательностей у нас возникли проблемы с их подачей в слой эмбеддингов. В качестве решения тут можно предложить следующий подход — сначала мы индексы "набьем" до одинаковой длины, затем получившийся тензор запакуем, и его пропустим через RNN. Посмотрим на реализацию такого смешанного подхода:

In [34]:
# Набивка индексов
texts_idxs_padded = pad_sequence(
    map(torch.LongTensor, texts_idxs),
    padding_value=word2ind[PAD_TOKEN]
)

# Получение эмбеддингов
embeds_padded = embeddings(texts_idxs_padded)

# Упаковка эмбеддингов
embeds_packed = pack_padded_sequence(
    embeds_padded,
    lengths=list(map(len, texts)),
    enforce_sorted=False
)

# Пропуск эмбеддингов через RNN
rnn = torch.nn.RNN(d_in, d_hidden)

rnn_out, h = rnn(embeds_packed)
rnn_out

PackedSequence(data=tensor([[ 0.2969,  0.0939,  0.3407, -0.1604, -0.2229,  0.2013, -0.0252,  0.0296,
         -0.2661,  0.3810],
        [ 0.3717,  0.0758,  0.3677, -0.1579, -0.2230,  0.1112,  0.0445, -0.0629,
         -0.3145,  0.5421],
        [-0.3036,  0.2989,  0.7541,  0.4138,  0.3406,  0.2465, -0.3178, -0.3097,
          0.3117,  0.3074],
        [ 0.4729,  0.0193,  0.1116, -0.3690, -0.4167,  0.2090,  0.0687,  0.1838,
         -0.4462,  0.3584],
        [ 0.3263,  0.0642,  0.1244, -0.3245, -0.3723,  0.3339, -0.0572,  0.2779,
         -0.3376,  0.0761],
        [ 0.5790, -0.0772,  0.0796, -0.2117, -0.4743, -0.1117,  0.3205, -0.1824,
         -0.5172,  0.6852],
        [ 0.3036,  0.0659,  0.0411, -0.0486, -0.4392,  0.1308,  0.0639,  0.0254,
         -0.2416,  0.2078],
        [ 0.7146, -0.0515, -0.3970, -0.0991, -0.6928,  0.0744, -0.0771,  0.6888,
         -0.7547,  0.1028],
        [-0.1044,  0.3376,  0.6206,  0.3965,  0.0202, -0.0022,  0.0065, -0.5795,
          0.4012,  0.5729],

Стоит отметить, что в силу своей природы RNN обрабатывает приходящие последовательности последовательно (в цикле), из-за чего оптимизация с помощью упаковки последовательности работает. [Трансформер](https://arxiv.org/abs/1706.03762) же в свою очередь обрабатывает одновременно всю последовательность целиком (из-за слоев self-attention), поэтому реализация метода упаковки для трансформера имеет свои сложности, и в `torch` этот подход не реализован. При работе с трансформером приходится работать с помощью метода набивки.

#### Работа загрузчика данных в `torch` с текстами

Для работы с данными в `torch` используют две сущности — набор данных (data set) и загрузчик данных (data loader). При работе с набором текстов в загрузчике данных стоит указывать `collate_fn` для склейки нескольких предложений в один батч. Для демонстрации этого давайте сначал сконструируем игрушечный набор данных:

In [35]:
import random
from random import randint, choices
from string import ascii_lowercase

from torch.utils.data import Dataset, DataLoader


class MyDataset(Dataset):
    """Набор данных из n случайных предложений"""
    def __init__(self, *, n=10, vocab_size):
        """
        n — число генерируемых предложений,
        vocab_size — размер словаря.
        """
        super().__init__()

        self.n = n
        random.seed(n)  # Для воспроизводимости экспериментов
        self.texts = [
            torch.LongTensor([
                # Вместо слова генерируем его индекс как
                #  будто бы мы уже перевели все слова в индексы
                randint(0, vocab_size - 1)
                for _ in range(randint(1, 8))
            ])
            for _ in range(n)
        ]

    def __len__(self):
        return self.n

    def __getitem__(self, ind):
        return self.texts[ind]

Посмотрим, что лежит в датасете:

In [36]:
dataset = MyDataset(vocab_size=len(vocabulary))
for x in dataset:
    print(x)

tensor([6])
tensor([ 9,  0,  3,  7, 13,  7, 13,  4])
tensor([0, 8, 7])
tensor([ 1,  3, 11,  5,  0,  6])
tensor([9, 5, 6])
tensor([ 4, 13, 10,  4,  7,  2, 10])
tensor([10,  5,  2,  7, 12])
tensor([7, 9, 6, 0])
tensor([3])
tensor([3, 4, 8])


Если мы теперь попробуем создать загрузчик данных от такого набора данных и начнем его использовать, то код упадет:

In [37]:
dataloader = DataLoader(dataset, batch_size=3)

try:
    batch = next(iter(dataloader))
except RuntimeError as e:
    print(f"Caught error: {e}")

Caught error: stack expects each tensor to be equal size, but got [1] at entry 0 and [8] at entry 1


Ошибка красноречива — предложения в батче оказались разной длины и загрузчик данных не знает, что с ними делать. Проблема в том, что для создания батча из отдельных его элементов `DataLoader` вызывает функцию `collate_fn` (collate — собирать, объединять). По умолчанию она просто склеивает поданные ей тензоры в один, поэтому тензоры разных размеров и вызывают ошибку. Решение тут простое — нужно задать свою функцию `collate_fn`.

Мы уже прошли два подхода для работы с предложениями разной длины, тогда и напишем две разных `collate_fn` функции:

* С запаковкой предложений:

In [38]:
def collate_pack(batch):
    """batch — список тензоров"""
    return pack_sequence(batch, enforce_sorted=False)


dataloader = DataLoader(dataset, batch_size=3, collate_fn=collate_pack)

for batch in dataloader:
    print(batch)
    print()

PackedSequence(data=tensor([ 9,  0,  6,  0,  8,  3,  7,  7, 13,  7, 13,  4]), batch_sizes=tensor([3, 2, 2, 1, 1, 1, 1, 1]), sorted_indices=tensor([1, 2, 0]), unsorted_indices=tensor([2, 0, 1]))

PackedSequence(data=tensor([ 4,  1,  9, 13,  3,  5, 10, 11,  6,  4,  5,  7,  0,  2,  6, 10]), batch_sizes=tensor([3, 3, 3, 2, 2, 2, 1]), sorted_indices=tensor([2, 0, 1]), unsorted_indices=tensor([1, 2, 0]))

PackedSequence(data=tensor([10,  7,  3,  5,  9,  2,  6,  7,  0, 12]), batch_sizes=tensor([3, 2, 2, 2, 1]), sorted_indices=tensor([0, 1, 2]), unsorted_indices=tensor([0, 1, 2]))

PackedSequence(data=tensor([3, 4, 8]), batch_sizes=tensor([1, 1, 1]), sorted_indices=tensor([0]), unsorted_indices=tensor([0]))



* И с набивкой предложений:

In [39]:
def collate_pad(batch):
    """batch — список тензоров"""
    return pad_sequence(batch, batch_first=True,
                        padding_value=word2ind[PAD_TOKEN])


dataloader = torch.utils.data.DataLoader(dataset, batch_size=3, collate_fn=collate_pad)

for batch in dataloader:
    print(batch)
    print()

tensor([[ 6, 13, 13, 13, 13, 13, 13, 13],
        [ 9,  0,  3,  7, 13,  7, 13,  4],
        [ 0,  8,  7, 13, 13, 13, 13, 13]])

tensor([[ 1,  3, 11,  5,  0,  6, 13],
        [ 9,  5,  6, 13, 13, 13, 13],
        [ 4, 13, 10,  4,  7,  2, 10]])

tensor([[10,  5,  2,  7, 12],
        [ 7,  9,  6,  0, 13],
        [ 3, 13, 13, 13, 13]])

tensor([[3, 4, 8]])



**Резюме главы**
1. Существует два похода для работы с батчом последовательностей разной длины: метод упаковки и метод набивки. Оба подхода реализованы в `torch`.
2. Метод упаковки позволяет делать меньше вычислений при работе с RNN.
3. Предварительная сортировка выборки по убыванию длин последовательностей поможет еще сильнее повысить эффективность работы RNN.
4. Упаковка последовательностей в `torch` реализована только для RNN.
5. При работе с текстами в загрузчиках данных стоит указывать `collate_fn` для правильного образования батчей.

[К оглавлению](#toc)

## Трансформеры <a id='section_transformers'></a>

### Self Attention (SA) <a id='ssection_transformers_SA'></a>
Основной частью трансформера является слой с механизмом самовнимания (Self Attention, SA). Давайте рассмотрим подробнее, как он работает.

![Self Attention](images/transformer_self_attention_vectors.png "Self Attention")

Слой SA на вход принимает последовательность эмбеддингов токенов, возвращает этот слой последовательность эмбеддингов такой же длины, но вообще говоря другой размерности.

На рисунке выше изображен вход слоя SA и некоторые из его параметров.
В этом примере слою подали эмбеддинги $x_1, \dots, x_n \in \mathbb{R}^{d_{emb}}$ (в примере $d_{emb} = 4, n = 2$), изображенные параметры слоя — $W^Q, W^K, W^V \in \mathbb{R}^{d_{emb} \times d_{SA}}$ (в примере $d_{SA} = 3$). К входным эмбеддингам $x_1, \dots x_n$ применяются аффинные преобразования с помощью вышеупомянутых матриц.
Так для каждого входного эмбеддинга $x_i$ получается набор из трех эмбеддингов — "запрос" $q_i = x_i W^Q$, "ключ" $k_i = x_i W^K$, "значение" $v_i = x_i W^V$ ($q_i, k_i, v_i \in \mathbb{R}^{d_{SA}}$). Для каждого запроса $q_i$ мы сейчас будем вычисляется значение ответа $z_i \in \mathbb{R}^{d_{SA}}$ — именно последовательность $z_1, \dots, z_n$ будет результатом работы алгоритма SA.

Рассмотрим вычисление $z_i$. Для этого запрос $q_i$ скалярно перемножается на все ключи $k_j, j = \overline{1, n}$, каждое скалярное произведение затем делится на $\sqrt{d_{SA}}$ для более хорошего обучения модели. Результат этой операции обозначим за:
$$
\text{sim}(q_i, k_j) = \frac{\langle q_i, k_j \rangle}{\sqrt{d_{SA}}}.
$$
По сути на этом шаге мы посчитали близость вектора $q_i$ со всеми векторами $k_j, j = \overline{1, n}$.

Далее этот набор близостей переводится в вероятностное распределение над всеми ключами с помощью преобразования $\text{softmax}$:
$$
\alpha_{ij} = \text{softmax} \left( \text{sim}(q_i, k_j) \right)_j = \frac{\exp(\text{sim}(q_i, k_j))}{\sum_{s=1}^{n} \exp(\text{sim}(q_i, k_s))}.
$$
Все $\alpha_{ij}$ не меньше $0$, а их сумма по $j$ от $1$ до $n$ дает $1$.

![Softmax in Self Attention](images/self-attention_softmax.jpg "Softmax in Self Attention")
На рисунке выше изображен подсчет скалярных произведений с последующих вычислением softmax для $q_1$. На рисунке за $d_k$ обозначается $d_{SA}$.

На последнем шаге алгоритма выход $z_i$ формируется в виде выпуклой комбинации векторов $v_j, j = \overline{1, n}$ с весами $\alpha_{ij}$:
$$
z_i = \sum_{j = 1}^{n} \alpha_{ij} v_j.
$$

![Выход слоя Self Attention](images/self-attention-output.jpg "Выход слоя Self Attention")
На рисунке выше изображен подсчет $z_1$.

Логику слоя Self Attention вкратце можно описать так: для каждого входного эмбеддинга $x_i$ мы определяем его близость со всеми входными словами, и результат для $x_i$ — сумма эмбеддингов входных слов с учетом их близости с $x_i$.

### Multi-head Self-Attention (MHSA) <a id='ssection_transformers_MHSA'></a>
Для того, чтобы улучшить работу слоя самовнимания применяют его модифицированную версию — многоголовое самовнимание (Multi-head Self-Attention, MHSA).

В этом слое также на вход приходят эмбеддинги $x_1, \dots, x_n \in \mathbb{R}^{d_{emb}}$, на выходе будет последовательность $z_1, \dots, z_n \in \mathbb{R}^{d_{emb}}$.
Этот слой по сути представляет из себя $N$ отдельных слоев самовнимания $SA^s$ ($N$ — гиперпараметр слоя MHSA), каждый из которых имеет параметры $W^Q_s, W^K_s, W^V_s \in \mathbb{R}^{d_{emb} \times d_{SA}}, s = \overline{1, N}$. Сам слой MHSA имеет еще дополнительный параметр $W^O \in \mathbb{R}^{N d_{SA} \times d_{emb}}$.

Для каждого входного токена $x_i$ считаются выходы $N$ SA-слоев: $\tilde{z}_i^s \in \mathbb{R}^{d_{SA}}, s = \overline{1, N}$. Далее все эти векторы конкатенируются, в результате чего получается вектор $\tilde{z}_i \in \mathbb{R}^{N d_{SA}}$. На последнем шаге вектор $\tilde{z}_i$ преобразуется с помощью $W^O$ в вектор $z_i \in \mathbb{R}^{d_{emb}}$:
$$
z_i = \tilde{z}_i W^O.
$$

![MHSA](images/transformer_multi-headed_self-attention-recap.png "MHSA")
На рисунке выше показана работа слоя MHSA в матричном виде.

На практике часто выбирают $N$ и $d_{SA}$ так, чтобы $N d_{SA} = d_{emb}$, тогда можно не использовать последнее преобразование $W^O$, ведь размерность $\tilde{z}_i$ в этом случае уже будет равна $d_{emb}$.

Подробнее про устройство всего трансформера можно почитать в статье ["The Illustrated Transformer"](https://jalammar.github.io/illustrated-transformer/).

В `torch` слой MHSA реализован в виде класса `torch.nn.MultiheadAttention`.

**Резюме главы**
1. Был рассмотрен базовый блок трансформера — слой многоголового самовнимания (MHSA)

[К оглавлению](#toc)

## Специальные токены <a id='section_special_tokens'></a>

В прошлых главах у нас зашла речь о специальных токенах. Мы уже успели познакомиться с токенами `<UNK>` и `<PAD>`. Оказывается, в NLP есть и другие часто используемые спецтокены:

* `<BOS>`, begin of sequence — начало последовательности. Используется при генерации текстов в качестве начального токена, с которого начинается генерация.
* `<EOS>`, end of sequence — конец предложения. Используется при генерации текстов в качестве последнего токена, которым заканчивается генерация, а также для того, чтобы модель могла нам сообщить о том, что она закончила генерацию.

![BOS and EOS tokens](images/bos-eos-tokens.pbm "BOS and EOS tokens")

* `<MASK>` — используется в BERT-подобных моделях для маскирования токенов в тексте для задачи MLM — Masked-Language Modeling. Суть задачи заключается в том, чтобы модель научилась предсказывать замаскированные слова по их контексту.
* `<CLS>` — используется в BERT-подобных моделях для решения задачи классификации всего предложения и заменяет собой пулинг эмбеддингов токенов предложения.

![MASK and CLS tokens](images/mask-cls-tokens.png "MASK and CLS tokens")

* `<SEP>` — используется в BERT-подобных моделях для разделения двух предложений для задачи NSP — Next Sentence Prediction. Суть задачи заключается в том, чтобы модель научилась  определять, идут ли два предложения в тексте друг за другом или нет.

![SEP token](images/sep-token.png "SEP token")

Специальные токены могут иметь разный вид и не обязательно записываются в треугольных скобках. Например, спецтокены могут записывать в квадратных скобках, но сути дела это не меняет.

**Резюме главы**
1. При обработке естественного языка часто используют специальные токены, которые позволяют решать определенные задачи.

[К оглавлению](#toc)